<a href="https://colab.research.google.com/github/ginkgo-ml/classification/blob/main/Tower15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#ToDO
#Lamp and flare purchase and handling
#Elemental strength potions
#Other potions
#Scrolls
#Spells
#Balance rooms
#Status effects - Blind, poison, hunger, confusion, berserk, mute, sticky hands
#Library rooms?
#Use legendary weapons/armor
#More legendary weapons/armor?
#End goal
#Add treasures with benefit
#Add walls

import random
import math
import os # Import os to clear the console for better game experience
from collections import deque

def new_grid(grid_rows, grid_cols, wall_char):
    return [[wall_char for _ in range(grid_cols)] for _ in range(grid_rows)]

def count_room_cells(grid, floor_char):
    room_cell_count = 0
    for r in range(len(grid)):
        for c in range(len(grid[0])):
            if grid[r][c] == floor_char:
                room_cell_count += 1
    return room_cell_count

def does_path_not_exist(start_r, start_c, end_r, end_c, grid, floor_char):
    rows = len(grid)
    cols = len(grid[0])
    visited = set()
    queue = deque([(start_r, start_c)])
    visited.add((start_r, start_c))

    if not (0 <= start_r < rows and 0 <= start_c < cols and grid[start_r][start_c] == floor_char):
        return True
    if not (0 <= end_r < rows and 0 <= end_c < cols and grid[end_r][end_c] == floor_char):
        return True

    while queue:
        r, c = queue.popleft()

        if (r, c) == (end_r, end_c):
            return False

        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

        for dr, dc in directions:
            nr, nc = r + dr, c + dc

            if 0 <= nr < rows and 0 <= nc < cols and \
               grid[nr][nc] == floor_char and (nr, nc) not in visited:
                visited.add((nr, nc))
                queue.append((nr, nc))

    return True

def drunk_carve(current_row=0, current_col=0, num_steps=1, grid=[], grid_rows_param=0, grid_cols_param=0, floor_char_param='.'):
  for _ in range(num_steps):
      # Ensure current_row/col are within bounds before marking as floor
      if 0 <= current_row < grid_rows_param and 0 <= current_col < grid_cols_param:
          grid[current_row][current_col] = floor_char_param
      else:
          # If initial current_row/col are out of bounds, try to find a valid starting point
          # or handle this edge case. For now, we'll let it proceed with best effort.
          pass

      direction = random.randint(0, 3)
      new_row, new_col = current_row, current_col

      if direction == 0:
          new_row -= 1
      elif direction == 1:
          new_row += 1
      elif direction == 2:
          new_col -= 1
      elif direction == 3:
          new_col += 1

      if 1 <= new_row < (grid_rows_param-1) and 1 <= new_col < (grid_cols_param-1):
          current_row, current_col = new_row, new_col
  return grid

def carve_grid(grid_rows_param, grid_cols_param, grid, floor_char_param, top_steps=0, bottom_steps=0, center_steps=0):
   # Ensure start/end points are valid before carving
   start_r, start_c = 1, 1
   end_r, end_c = grid_rows_param - 2, grid_cols_param - 2

   # Adjust start/end points if they are out of bounds for very small grids
   if start_r < 0: start_r = 0
   if start_c < 0: start_c = 0
   if end_r >= grid_rows_param: end_r = grid_rows_param - 1
   if end_c >= grid_cols_param: end_c = grid_cols_param - 1
   if end_r < 0: end_r = 0 # Handle 0 or 1 row grids
   if end_c < 0: end_c = 0 # Handle 0 or 1 col grids


   grid = drunk_carve(start_r, start_c, top_steps, grid, grid_rows_param, grid_cols_param, floor_char_param)
   grid = drunk_carve(end_r, end_c, bottom_steps, grid, grid_rows_param, grid_cols_param, floor_char_param)

   count=0
   # Only check path if start and end are valid and distinct
   if 0 <= start_r < grid_rows_param and 0 <= start_c < grid_cols_param and \
      0 <= end_r < grid_rows_param and 0 <= end_c < grid_cols_param and \
      (start_r, start_c) != (end_r, end_c):
       while does_path_not_exist(start_r, start_c, end_r, end_c, grid, floor_char_param) and count<4:
         grid = drunk_carve(int(grid_rows_param/2), int(grid_cols_param/2), 75, grid, grid_rows_param, grid_cols_param, floor_char_param)
         count+=1

   maxed = count<4

   return grid, maxed



class Room:
    def __init__(self, room_type, properties=None):
        self.room_type = room_type
        self.properties = properties if properties is not None else {}
        self.discovered = False

    def __repr__(self):
        return f"Room(type='{self.room_type}', properties={self.properties})"

class Floor:
    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.grid = [] # Will hold a rows x cols grid of Room objects

    def generate_carved_layout(self, wall_char, floor_char,
                                 top_steps=0, bottom_steps=0, center_steps=0):
        temp_char_grid = new_grid(self.rows, self.cols, wall_char)

        while True:
          carved_grid, maxed_attempts = carve_grid(self.rows, self.cols, temp_char_grid, floor_char,
                                                  top_steps, bottom_steps, center_steps)
          if maxed_attempts:
            break

        return carved_grid

    def populate_rooms(self, carved_layout_grid, specified_chars_for_fill, required_chars_to_place_randomly,
                       wall_char, floor_char, forced_placement_char=None, forced_placement_coords=None,
                       p_limits=(0, 100), c_limits=(0, 100), w_limits=(0, 100), a_limits=(0, 100)):
        self.grid = [[None for _ in range(self.cols)] for _ in range(self.rows)]
        available_floor_cells_initial = []

        for r in range(self.rows):
            for c in range(self.cols):
                char = carved_layout_grid[r][c]
                if char == wall_char:
                    self.grid[r][c] = Room(wall_char)
                elif char == floor_char:
                    available_floor_cells_initial.append((r, c))

        available_floor_cells_for_random = [coord for coord in available_floor_cells_initial if self.grid[coord[0]][coord[1]] is None]

        # Initialize counters for limited characters
        char_counts_map = {'P': 0, 'C': 0, 'W': 0, 'A': 0}
        char_limits_map = {'P': p_limits, 'C': c_limits, 'W': w_limits, 'A': a_limits}
        priority_chars = ['P', 'C', 'W', 'A']

        # Handle forced placement (Entrance 'E' or Downstairs 'D')
        if forced_placement_char and forced_placement_coords:
            r, c = forced_placement_coords
            if 0 <= r < self.rows and 0 <= c < self.cols and carved_layout_grid[r][c] == floor_char:
                self.grid[r][c] = Room(forced_placement_char)
                if (r, c) in available_floor_cells_for_random:
                    available_floor_cells_for_random.remove((r, c))
                # Update count if forced char is one of the limited types
                if forced_placement_char in char_counts_map:
                    char_counts_map[forced_placement_char] += 1
            else:
                print(f"Warning: Forced placement coordinates {forced_placement_coords} for '{forced_placement_char}' is not a floor cell or out of bounds. Attempting to find alternative.")
                found_alt = False
                for alt_r, alt_c in list(available_floor_cells_for_random):
                    if self.grid[alt_r][alt_c] is None:
                        self.grid[alt_r][alt_c] = Room(forced_placement_char)
                        available_floor_cells_for_random.remove((alt_r, alt_c))
                        found_alt = True
                        # Update count if forced char is one of the limited types
                        if forced_placement_char in char_counts_map:
                            char_counts_map[forced_placement_char] += 1
                        break
                if not found_alt:
                    raise ValueError(f"Could not find a valid floor cell for forced placement of '{forced_placement_char}'.")

        # Place explicitly required_chars (U,V) into distinct *random* available floor cells
        random.shuffle(available_floor_cells_for_random)

        for char_type in required_chars_to_place_randomly:
            if available_floor_cells_for_random:
                r, c = available_floor_cells_for_random.pop(0)
                self.grid[r][c] = Room(char_type)
                # Update count if this char is one of the limited types
                if char_type in char_counts_map:
                    char_counts_map[char_type] += 1
            else:
                print(f"Warning: Not enough *empty* floor cells to place required character {char_type}.")

        # Prioritize placing characters to meet minimum requirements for P, C, W, A
        cells_to_process_for_min_fill = list(available_floor_cells_for_random)
        random.shuffle(cells_to_process_for_min_fill) # Shuffle to distribute min fills evenly

        for char_to_fill in priority_chars:
            min_count, _ = char_limits_map[char_to_fill]
            while char_counts_map[char_to_fill] < min_count and cells_to_process_for_min_fill:
                r, c = cells_to_process_for_min_fill.pop(0)
                if self.grid[r][c] is None: # Only place if cell is empty
                    self.grid[r][c] = Room(char_to_fill)
                    char_counts_map[char_to_fill] += 1
                    # Remove from original available_floor_cells_for_random list as well
                    if (r, c) in available_floor_cells_for_random:
                        available_floor_cells_for_random.remove((r, c))

        # Re-initialize available_floor_cells_for_random based on current empty spots
        available_floor_cells_for_random = [coord for coord in available_floor_cells_initial if self.grid[coord[0]][coord[1]] is None]
        random.shuffle(available_floor_cells_for_random)

        # Fill the remaining empty floor cells with random specified_chars respecting max limits
        excluded_from_fill = set(required_chars_to_place_randomly + [wall_char])
        if forced_placement_char:
            excluded_from_fill.add(forced_placement_char)

        for r, c in available_floor_cells_for_random: # Iterate over remaining empty cells
            if self.grid[r][c] is None: # Double-check if cell is still empty (should be)
                valid_fill_options = []
                for char in specified_chars_for_fill:
                    if char not in excluded_from_fill:
                        if char in priority_chars:
                            if char_counts_map[char] < char_limits_map[char][1]: # Check max limit dynamically
                                valid_fill_options.append(char)
                        else:
                            valid_fill_options.append(char) # Other chars (M, .) have no max limit

                if not valid_fill_options:
                    chosen_char = floor_char # Fallback if all other fill options reached max
                else:
                    chosen_char = random.choice(valid_fill_options)

                self.grid[r][c] = Room(chosen_char)
                if chosen_char in char_counts_map:
                    char_counts_map[chosen_char] += 1

        # Final check: Ensure all cells are filled
        for r in range(self.rows):
            for c in range(self.cols):
                if self.grid[r][c] is None:
                    self.grid[r][c] = Room(floor_char)

    def print_floor(self, highlight_coords=None):
        for r_idx in range(self.rows):
            for c_idx in range(self.cols):
                room = self.grid[r_idx][c_idx]
                if room.discovered or (r_idx, c_idx) == highlight_coords:
                    char_to_print = room.room_type
                else:
                    char_to_print = ' ' # Fog of war character
                output = f"[{char_to_print}]" if highlight_coords == (r_idx, c_idx) else f" {char_to_print} "
                print(output, end="")
            print()

    def print_floor_debug(self):
        """Prints the floor layout showing all room types, ignoring fog of war."""
        for r_idx in range(self.rows):
            for c_idx in range(self.cols):
                room = self.grid[r_idx][c_idx]
                char_to_print = room.room_type
                output = f" {char_to_print} "
                print(output, end="")
            print()

    def __repr__(self):
        grid_str = []
        for row in self.grid:
            row_str = " ".join([room.room_type for room in row])
            grid_str.append(row_str)
        return "Floor(\n" + "\n".join(grid_str) + "\n)"

class Tower:
    def __init__(self):
        self.floors = []

    def add_floor(self, specified_chars, required_chars, grid_rows, grid_cols,
                  wall_char, floor_char,
                  top_steps=75, bottom_steps=75, center_steps=75,
                  p_limits=(0, 100), c_limits=(0, 100), w_limits=(0, 100), a_limits=(0, 100)):
        new_floor = Floor(grid_rows, grid_cols)

        carved_layout_grid = new_floor.generate_carved_layout(
            wall_char, floor_char, top_steps, bottom_steps, center_steps
        )

        forced_placement_char = None
        forced_placement_coords = None

        required_for_random_placement = [char for char in required_chars if char not in ['D', 'E']]

        current_fill_pool = [char for char in specified_chars if char not in ['E', 'D']]
        if floor_char not in current_fill_pool:
            current_fill_pool.append(floor_char)


        if not self.floors: # First floor
            forced_placement_char = 'E'

            if 0 <= 0 < grid_rows and 0 <= 1 < grid_cols and carved_layout_grid[0][1] == floor_char:
                forced_placement_coords = (0,1)
            else:
                for r in range(grid_rows):
                    for c in range(grid_cols):
                        if carved_layout_grid[r][c] == floor_char:
                            forced_placement_coords = (r, c)
                            break
                    if forced_placement_coords:
                        break
            if not forced_placement_coords:
                raise ValueError("Could not find a floor cell to place the entrance 'E'.")
            print(f"Generated first floor with 'E' at {forced_placement_coords}.")

        else: # Subsequent floors
            forced_placement_char = 'D'

            if 0 <= grid_rows-2 < grid_rows and 0 <= grid_cols-2 < grid_cols and carved_layout_grid[grid_rows-2][grid_cols-2] == floor_char:
                 forced_placement_coords = (grid_rows-2, grid_cols-2)
            else:
                for r in range(grid_rows - 1, -1, -1):
                    for c in range(grid_cols - 1, -1, -1):
                        if carved_layout_grid[r][c] == floor_char:
                            forced_placement_coords = (r, c)
                            break
                    if forced_placement_coords:
                        break
            if not forced_placement_coords:
                raise ValueError("Could not find a floor cell to place the downstairs 'D'.")

        new_floor.populate_rooms(carved_layout_grid,
                                 current_fill_pool,
                                 required_for_random_placement,
                                 wall_char, floor_char,
                                 forced_placement_char=forced_placement_char,
                                 forced_placement_coords=forced_placement_coords,
                                 p_limits=p_limits, c_limits=c_limits, w_limits=w_limits, a_limits=a_limits)

        self.floors.append(new_floor)

    def __repr__(self):
        return f"Tower(number_of_floors={len(self.floors)})"



#---- List of elements ----
ELEMENTS = ['Darkness', 'Demonic', 'Earth', 'Fire', 'Holy', 'Ice', 'Light', 'Physical', 'Psionic', 'Water', 'Wind']

# --- Item Classes ---
class Item:
    def __init__(self, name, description="", value=0, level=0):
        self.name = name
        self.description = description
        self.value = value # Added a value attribute for buying/selling
        self.level = level # Added a level attribute

    def __repr__(self):
        return f"Item(name='{self.name}', description='{self.description}', value={self.value}, level={self.level})"

    def use(self, character, my_tower=None):
        print(f"Using {self.name}. (No specific effect defined for base item)")
        return False # Base item generally not consumed

class Potion(Item):
    def __init__(self, name, description="", healing_amount=0, potion_type='healing', stat_to_boost=None, boost_amount=0, experience_gain=0, value=0, level=0):
        super().__init__(name, description, value, level)
        self.healing_amount = healing_amount
        self.potion_type = potion_type
        self.stat_to_boost = stat_to_boost
        self.boost_amount = boost_amount
        self.experience_gain = experience_gain

    def __repr__(self):
        if self.potion_type == 'healing':
            return f"Potion(name='{self.name}', type='{self.potion_type}', healing={self.healing_amount}, value={self.value}, level={self.level})"
        elif self.potion_type == 'stat_boost':
            return f"Potion(name='{self.name}', type='{self.potion_type}', boosts='{self.stat_to_boost}' by {self.boost_amount}, value={self.value}, level={self.level})"
        elif self.potion_type == 'experience':
            return f"Potion(name='{self.name}', type='{self.potion_type}', exp_gain={self.experience_gain}, value={self.value}, level={self.level})"
        else:
            return f"Potion(name='{self.name}', type='{self.potion_type}', value={self.value}, level={self.level})"

    def use(self, character, my_tower=None):
        if self.potion_type == 'healing':
            if character.health < 100: # Assuming max health is 100 for simplicity
                character.health += self.healing_amount
                if character.health > 100:
                    character.health = 100
                print(f"{character.name} used {self.name} and healed for {self.healing_amount} HP. Current health: {character.health})")
                return True # Consumed
            else:
                print(f"{character.name}'s health is already full. {self.name} had no effect.")
                return False # Not consumed
        elif self.potion_type == 'stat_boost':
            if self.stat_to_boost and hasattr(character, self.stat_to_boost):
                current_stat_value = getattr(character, self.stat_to_boost)
                setattr(character, self.stat_to_boost, current_stat_value + self.boost_amount)
                print(f"{character.name} used {self.name} and {self.stat_to_boost} increased by {self.boost_amount}. New {self.stat_to_boost}: {getattr(character, self.stat_to_boost)})")
                return True # Consumed
            else:
                print(f"Cannot boost stat: {self.stat_to_boost} for {character.name}. {self.name} had no effect.")
                return False # Not consumed
        elif self.potion_type == 'experience':
            character.experience += self.experience_gain
            print(f"{character.name} used {self.name} and gained {self.experience_gain} experience. Total experience: {character.experience})")
            return True # Consumed
        else:
            print(f"Unknown potion type: {self.potion_type}. {self.name} had no effect.")
            return False # Not consumed

class Weapon(Item):
    def __init__(self, name, description="", attack_bonus=0, value=0, level=0, upgrade_level=0, elemental_strength=["None"], upgrade_limit=True):
        super().__init__(name, description, value, level)
        self._base_attack_bonus = attack_bonus
        self.upgrade_level = upgrade_level

    @property
    def attack_bonus(self):
        return self._base_attack_bonus + self.upgrade_level

    def __repr__(self):
        return f"Weapon(name='{self.name}', attack_bonus={self.attack_bonus}, value={self.value}, level={self.level}, upgrade_level={self.upgrade_level})"

class Armor(Item):
    def __init__(self, name, description="", defense_bonus=0, value=0, level=0, upgrade_level=0, elemental_strength=["None"], upgrade_limit=True):
        super().__init__(name, description, value, level)
        self._base_defense_bonus = defense_bonus
        self.upgrade_level = upgrade_level

    @property
    def defense_bonus(self):
        return self._base_defense_bonus + self.upgrade_level

    def __repr__(self):
        return f"Armor(name='{self.name}', defense_bonus={self.defense_bonus}, value={self.value}, level={self.level}, upgrade_level={self.upgrade_level})"

class Scroll(Item):
    def __init__(self, name, description="", effect_description="", value=0, level=0, scroll_type='generic'):
        super().__init__(name, description, value, level)
        self.effect_description = effect_description
        self.scroll_type = scroll_type # New attribute for scroll type

    def __repr__(self):
        return f"Scroll(name='{self.name}', effect='{self.effect_description}', type='{self.scroll_type}', value={self.value}, level={self.level})"

    def use(self, character, my_tower=None):
        if self.scroll_type == 'upgrade':
            print(f"{character.name} reads the {self.name}. You feel its magic seeking an item to enhance...")

            upgradable_items = []
            for item in character.inventory.items:
                if isinstance(item, (Weapon, Armor)):
                    upgradable_items.append(item)

            if not upgradable_items:
                print("You have no weapons or armor in your inventory that can be upgraded.")
                return False # Scroll not consumed

            print("\nSelect an item to upgrade:")
            for i, item in enumerate(upgradable_items):
                if isinstance(item, Weapon):
                    print(f"  {i + 1}. {item.name} (Attack: {item.attack_bonus}, +{item.upgrade_level} Upgrade Level)")
                elif isinstance(item, Armor):
                    print(f"  {i + 1}. {item.name} (Defense: {item.defense_bonus}, +{item.upgrade_level} Upgrade Level)")

            while True:
                choice = input("Enter the number of the item you wish to upgrade (or 'cancel'): ").lower()

                if choice == 'cancel':
                    print("Upgrade cancelled. The scroll remains unused.")
                    return False # Scroll not consumed

                try:
                    item_index = int(choice) - 1
                    if 0 <= item_index < len(upgradable_items):
                        chosen_item = upgradable_items[item_index]
                        if chosen_item.upgrade_limit:
                          if item.upgrade_level >= (item.level+1):
                            upgrade_prob = .25+(player_character.intelligence/100)+(player_character.level/100)
                          else:
                            upgrade_prob = .75+(player_character.intelligence/100)+(player_character.level/100)
                        else:
                          upgrade_prob = .75+(player_character.intelligence/100)+(player_character.level/100)
                        if random.random() < upgrade_prob:
                          chosen_item.upgrade_level += 1
                          print(f"Successfully upgraded {chosen_item.name}! It is now +{chosen_item.upgrade_level}.")
                          return True # Scroll consumed
                        else:
                          print("Upgrade failed. The scroll crumbles in your hands.")
                          return True # Scroll not consumed
                    else:
                        print("Invalid number. Please choose a number from the list.")
                except ValueError:
                    print("Invalid input. Please enter a number or 'cancel'.")
        else:
            # Generic scroll behavior
            print(f"{character.name} read the {self.name}. Effect: {self.effect_description}")
            return True # Scroll consumed

class Flare(Item):
    def __init__(self, name, description="", duration_rounds=3, light_radius=5, value=0, level=0):
        super().__init__(name, description, value, level)
        self.duration_rounds = duration_rounds
        self.light_radius = light_radius

    def __repr__(self):
        return f"Flare(name='{self.name}', duration={self.duration_rounds} rounds, radius={self.light_radius}, value={self.value}, level={self.level})"

    def use(self, character, my_tower):
        print(f"{character.name} lit the {self.name}.")
        if my_tower is None:
            print("Error: Flare cannot reveal rooms without tower and grid information.")
            return True # Still consumed

        while True:
            direction = input("Shine the flare in a direction (n, s, e, w, or 'cancel'): ").lower()
            if direction == 'cancel':
                print("Flare not shined in any specific direction, but it is consumed.")
                break # Flare is still consumed

            if direction in ['n', 's', 'e', 'w']:
                target_x, target_y = move_player(character, my_tower, direction)
                if (target_x, target_y) == (character.x, character.y):
                    print("Cannot shine the flare further in that direction (boundary reached). Choose another direction or 'cancel'.")
                    continue

                # Reveal the target room
                target_floor = my_tower.floors[character.z]
                target_room = target_floor.grid[target_x][target_y]
                target_room.discovered = True
                print(f"The flare reveals the room at ({target_x}, {target_y}) on Floor {character.z + 1}.")
                print(f"It contains a '{target_room.room_type}'.")
                break # Flare consumed and effect applied
            else:
                print("Invalid direction. Please enter n, s, e, w, or 'cancel'.")
        return True # Flare is consumed upon use

class Lantern(Item):
    def __init__(self, name, description="", fuel_amount=10, light_radius=7, value=0, level=0):
        super().__init__(name, description, value, level)
        self.fuel_amount = fuel_amount
        self.light_radius = light_radius

    def __repr__(self):
        return f"Lantern(name='{self.name}', fuel={self.fuel_amount}, radius={self.light_radius}, value={self.value}, level={self.level})"

    def use(self, character, my_tower=None):
        if self.fuel_amount > 0:
            print(f"{character.name} lit the {self.name}. It illuminates a radius of {self.light_radius}.")

            if my_tower is None:
                print("Error: Lantern cannot reveal rooms without tower and grid information.")
                return False # Not consumed

            current_floor = my_tower.floors[character.z]
            directions_to_reveal = [
                (-1, 0), (1, 0), (0, -1), (0, 1),  # Cardinal (N, S, W, E)
                (-1, -1), (-1, 1), (1, -1), (1, 1) # Intermediate (NW, NE, SW, SE)
            ]

            revealed_any = False
            for dr, dc in directions_to_reveal:
                target_x, target_y = character.x + dr, character.y + dc

                # Check boundaries
                if 0 <= target_x < current_floor.cols and 0 <= target_y < current_floor.rows:
                    target_room = current_floor.grid[target_x][target_y]
                    if not target_room.discovered:
                        target_room.discovered = True
                        print(f"The lantern reveals the room at ({target_x}, {target_y}). It contains a '{target_room.room_type}'.")
                        revealed_any = True

            if not revealed_any:
                print("The lantern shines brightly, but no new undiscovered rooms were revealed nearby.")

            self.fuel_amount -= 1
            print(f"Fuel remaining: {self.fuel_amount})")
            return False # Lantern is not consumed, only fuel
        else:
            print(f"The {self.name} is out of fuel.")
            return False # Not consumed, already out of fuel

class Treasure(Item):
    def __init__(self, name, description="", gold_value=0, value=0, benefit=None, level=0):
        super().__init__(name, description, value, level)
        self.gold_value = gold_value
        self.benefit = benefit if benefit is not None else {}

    def __repr__(self):
        return f"Treasure(name='{self.name}', value={self.gold_value} gold, benefit={self.benefit}, level={self.level})"

    def collect(self, character):
        character.gold += self.gold_value
        print(f"{character.name} collected {self.name} worth {self.gold_value} gold. Total gold: {character.gold})")
        return True # Treasure is consumed upon collection

# --- Inventory Class ---
class Inventory:
    def __init__(self):
        self.items = [] # Will store actual Item objects

    def add_item(self, item_obj): # Accepts an Item object
        self.items.append(item_obj)
        print(f"Added {item_obj.name} to inventory.")

    def add_item_quiet(self, item_obj): # Accepts an Item object
        self.items.append(item_obj)
        #print(f"Added {item_obj.name} to inventory.")

    def remove_item(self, item_name): # Accepts item name to remove one instance
        found = False
        for item in self.items:
            if item.name.lower() == item_name.lower():
                self.items.remove(item)
                found = True
                print(f"Removed {item_name} from inventory.")
                break
        if not found:
            print(f"{item_name} not found in inventory.")
        return found # Return if item was successfully removed

    def get_item(self, item_name): # New method to retrieve an item object by name
        for item in self.items:
            if item.name.lower() == item_name.lower():
                return item
        return None

    def check_item(self, item_name): # New method to check an item object by name
        for item in self.items:
            if item.name.lower() == item_name.lower():
                return True
        return False

    def display_inventory(self):
        if not self.items:
            print("Inventory is empty.")
        else:
            print("Inventory:")
            for i, item in enumerate(self.items):
                item_display = f"  {i + 1}. {item.name} (Value: {item.value} gold)"
                if isinstance(item, (Weapon, Armor)):
                    item_display += f" +{item.upgrade_level}"
                print(item_display)

    def __repr__(self):
        return f"Inventory(items={[item.name for item in self.items]})"

# --- Character Class ---
class Character:
    def __init__(self, name, health, attack, defense, strength, dexterity, intelligence, level=1, gold=0, experience=0, x=0, y=0, z=0, character_class="Adventurer", race="Human", gender="Unknown", elemental_weakness=["None"], elemental_strength=["None"]):
        self.name = name
        self.health = health
        self._base_attack = attack # Store base attack
        self._base_defense = defense # Store base defense
        self.strength = strength
        self.dexterity = dexterity
        self.intelligence = intelligence
        self._base_level = level
        self.gold = gold
        self.experience = experience
        self.x = x
        self.y = y
        self.z = z # Represents the current floor level in the tower
        self.character_class = character_class
        self.race = race
        self.gender = gender
        self.inventory = Inventory() # Each character gets their own inventory
        self.equipped_weapon = None
        self.equipped_armor = None
        self.elemental_weakness = elemental_weakness
        self.elemental_strength = elemental_strength

    #add level to other characteristics

    @property
    def level(self):
      add_lvl = int(math.sqrt(self.experience/5))
      return self._base_level + add_lvl

    @property
    def attack(self):
        bonus = self.equipped_weapon.attack_bonus if self.equipped_weapon else 0
        return self._base_attack + bonus

    @property
    def defense(self):
        bonus = self.equipped_armor.defense_bonus if self.equipped_armor else 0
        return self._base_defense + bonus

    def __repr__(self):
        equipped_weapon_name = self.equipped_weapon.name if self.equipped_weapon else "None"
        equipped_armor_name = self.equipped_armor.name if self.equipped_armor else "None"
        return f"Character(name='{self.name}', health={self.health}, attack={self.attack}, defense={self.defense}, strength={self.strength}, dexterity={self.dexterity}, intelligence={self.intelligence}, level={self.level}, gold={self.gold}, experience={self.experience}, x={self.x}, y={self.y}, z={self.z}, character_class='{self.character_class}', race='{self.race}', gender='{self.gender}', equipped_weapon='{equipped_weapon_name}', equipped_armor='{equipped_armor_name}', inventory={self.inventory})"

    def add_elemental_strength(self, element):
        if element in self.elemental_weakness:
           self.remove_elemental_weakness(element)
        if element not in self.elemental_strength:
           self.elemental_strength.append(element)
           print(f"{self.name} is now strong against {element}.")
        else:
          print(f"{self.name} is already strong against {element}.")

    def add_elemental_weakness(self, element):
        if element in self.elemental_strength:
           self.remove_elemental_strength(element)
        if element not in self.elemental_weakness:
           self.elemental_weakness.append(element)
           print(f"{self.name} is now weak against {element}.")
        else:
           print(f"{self.name} is already weak against {element}.")

    def remove_elemental_weakness(self, element):
        if element in self.elemental_weakness:
           self.elemental_weakness.remove(element)
           print(f"{self.name} is no longer weak against {element}.")
        else:
           print(f"{self.name} is not weak against {element}.")

    def remove_elemental_strength(self, element):
        if element in self.elemental_strength:
           self.elemental_strength.remove(element)
           print(f"{self.name} is no longer strong against {element}.")
        else:
           print(f"{self.name} is not strong against {element}.")

    def attack_target(self, target_character):
        #print(f"{self.name} attacks {target_character.name}!")
        # Attack damage now scales with monster level
        elemental = self.equipped_weapon.elemental_strength if self.equipped_weapon else "Empty"
        scaled_attack = self.attack
        if elemental in target_character.elemental_weakness:
          scaled_attack==int(scaled_attack*1.5)
          print(f"{target_character.name} is weak against {elemental}!")
        return scaled_attack

    def take_damage(self, damage, element="Empty"):
        if element in self.elemental_weakness:
          damage=int(damage*1.5)
          print(f"{self.name} is weak against {element}!")

        if element in self.elemental_strength:
          damage=int(damage*.5)
          print(f"{self.name} is strong against {element}!")

        actual_damage = max(0, damage - self.defense)
        self.health -= actual_damage
        if self.health < 0:
            self.health = 0
        print(f"{self.name} took {actual_damage} damage. Current health: {self.health})")
        return self.health == 0

    def take_damage_no_def(self, damage, element="Empty"):
        if element in self.elemental_weakness:
          damage=int(damage*1.5)
          print(f"{self.name} is weak against {element}!")

        if element in self.elemental_strength:
          damage=int(damage*.5)
          print(f"{self.name} is strong against {element}!")

        actual_damage = max(0, damage)
        self.health -= actual_damage
        if self.health < 0:
            self.health = 0
        print(f"{self.name} took {actual_damage} damage. Current health: {self.health})")
        return self.health == 0

    def is_alive(self):
        return self.health > 0

    def equip_item(self, item):
        if isinstance(item, Weapon):
            if self.equipped_weapon:
                print(f"Unequipping {self.equipped_weapon.name} and adding it back to inventory.")
                self.inventory.add_item(self.equipped_weapon)
            self.equipped_weapon = item
            print(f"{self.name} equipped {item.name}. Attack is now {self.attack}.")
            self.inventory.remove_item(item.name) # Remove from inventory after equipping
        elif isinstance(item, Armor):
            if self.equipped_armor:
                print(f"Unequipping {self.equipped_armor.name} and adding it back to inventory.")
                self.inventory.add_item(self.equipped_armor)
            self.equipped_armor = item
            print(f"{self.name} equipped {item.name}. Defense is now {self.defense}.")
            self.inventory.remove_item(item.name) # Remove from inventory after equipping
        else:
            print(f"Cannot equip {item.name}. It is not a weapon or armor.")

    def unequip_weapon(self):
        if self.equipped_weapon:
            print(f"Unequipping {self.equipped_weapon.name} and adding it back to inventory.")
            self.inventory.add_item(self.equipped_weapon)
            self.equipped_weapon = None
            print(f"Weapon unequipped. Attack is now {self.attack}.")
        else:
            print("No weapon equipped to unequip.")

    def unequip_armor(self):
        if self.equipped_armor:
            print(f"Unequipping {self.equipped_armor.name} and adding it back to inventory.")
            self.inventory.add_item(self.equipped_armor)
            self.equipped_armor = None
            print(f"Armor unequipped. Defense is now {self.defense}.")
        else:
            print("No armor equipped to unequip.")

class Monster:
    def __init__(self, name, health, attack, defense, elemental_weakness=["None"], elemental_strength=["None"], level=1):
        self.name = name
        self.health = health
        self.max_health = health # Store initial health as max health
        self.attack = attack
        self.defense = defense
        self.level = level
        self.elemental_weakness=elemental_weakness
        self.elemental_strength=elemental_strength

    def __repr__(self):
        return f"Monster(name='{self.name}', health={self.health}/{self.max_health}, attack={self.attack}, defense={self.defense}, level={self.level})"

    def add_elemental_strength(self, element):
        if element in self.elemental_weakness:
           self.remove_elemental_weakness(element)
        if element not in self.elemental_strength:
           self.elemental_strength.append(element)
           print(f"{self.name} is now strong against {element}.")
        else:
          print(f"{self.name} is already strong against {element}.")

    def add_elemental_weakness(self, element):
        if element in self.elemental_strength:
           self.remove_elemental_strength(element)
        if element not in self.elemental_weakness:
           self.elemental_weakness.append(element)
           print(f"{self.name} is now weak against {element}.")
        else:
           print(f"{self.name} is already weak against {element}.")

    def remove_elemental_weakness(self, element):
        if element in self.elemental_weakness:
           self.elemental_weakness.remove(element)
           print(f"{self.name} is no longer weak against {element}.")
        else:
           print(f"{self.name} is not weak against {element}.")

    def remove_elemental_strength(self, element):
        if element in self.elemental_strength:
           self.elemental_strength.remove(element)
           print(f"{self.name} is no longer strong against {element}.")
        else:
           print(f"{self.name} is not strong against {element}.")

    def take_damage(self, damage, element="Empty"):

        if element in self.elemental_weakness:
          damage=int(damage*1.5)
          print(f"{self.name} is weak against {element}!")

        if element in self.elemental_strength:
          damage=int(damage*.5)
          print(f"{self.name} is strong against {element}!")

        actual_damage = max(0, damage - self.defense)
        self.health -= actual_damage
        if self.health < 0:
            self.health = 0
        print(f"{self.name} took {actual_damage} damage. Current health: {self.health})")
        return self.health == 0

    def is_alive(self):
        return self.health > 0

    def attack_target(self, target_character):
        print(f"{self.name} attacks {target_character.name}!")
        # Attack damage now scales with monster level
        scaled_attack = self.attack * self.level
        target_character.take_damage(scaled_attack, self.elemental_strength)

def _get_potential_coords(start_x, start_y, direction):
    """
    Calculates potential new coordinates based on direction, without boundary checks.
    """
    new_x, new_y = start_x, start_y
    if direction == 'n': new_x -= 1
    elif direction == 's': new_x += 1
    elif direction == 'w': new_y -= 1
    elif direction == 'e': new_y += 1
    return new_x, new_y

def is_wall_at_coordinate(current_floor, r, c):
    # Ensure coordinates are within bounds
    if not (0 <= r < current_floor.rows and 0 <= c < current_floor.cols):
        #print("Out of bounds")
        return True

    return current_floor.grid[r][c].room_type == wall_char

def reveal_cells_around_character(character, my_tower, radius):
    """
    Reveals cells around the given character's position on the current floor.
    Changes discovered wall cells to floor characters for map display.
    """
    current_floor = my_tower.floors[character.z]
    revealed_any_new = False

    offsets = set()
    for i in range(1, radius + 1):
        offsets.add((i, 0))    # South
        offsets.add((-i, 0))   # North
        offsets.add((0, i))    # East
        offsets.add((0, -i))   # West

    for r_offset, c_offset in offsets:
        reveal_r, reveal_c = character.x + r_offset, character.y + c_offset
        if 0 <= reveal_r < current_floor.rows and 0 <= reveal_c < current_floor.cols:
            room_to_reveal = current_floor.grid[reveal_r][reveal_c]
            if not room_to_reveal.discovered and room_to_reveal.room_type == wall_char:
                room_to_reveal.discovered = True
                revealed_any_new = True
    return revealed_any_new

def move_player(player_character, my_tower, direction):
    """
    Attempts to move the player in the given direction.
    If successful, updates the player's coordinates and reveals adjacent cells.

    Args:
        player_character (Character): The player's character object.
        my_tower (Tower): The Tower object.
        direction (str): The direction of movement ('n', 's', 'e', 'w').

    Returns:
        bool: True if the player successfully moved, False otherwise (e.g., hit a wall).
    """
    current_floor = my_tower.floors[player_character.z]
    potential_new_x, potential_new_y = _get_potential_coords(player_character.x, player_character.y, direction)

    if is_wall_at_coordinate(current_floor, potential_new_x, potential_new_y):
        return player_character.x, player_character.y # Move blocked by wall or boundary
    else:
        #player_character.x, player_character.y = potential_new_x, potential_new_y
        #reveal_cells_around_character(player_character, my_tower, radius=1) # Reveal immediate surroundings after move
        return potential_new_x, potential_new_y

def move_player_randomly(player_character, my_tower, move_player_func_ref):
    """
    Moves the player in a random valid direction, updating player_character's position.

    Args:
        player_character (Character): The player's character object.
        my_tower (Tower): The Tower object.
        move_player_func_ref (function): The move_player function to use for movement calculation.

    Returns:
        tuple: A tuple (new_x, new_y) representing the player's new coordinates after the move.
    """
    current_x, current_y = player_character.x, player_character.y
    all_directions = ['n', 's', 'e', 'w']
    random.shuffle(all_directions) # Shuffle to attempt directions randomly

    for direction in all_directions:
        if move_player_func_ref(player_character, my_tower, direction): # Use the updated move_player function
            print(f"Moved {direction.upper()}. New position: ({player_character.x}, {player_character.y})")
            return player_character.x, player_character.y # Return updated coordinates

    print("No valid moves available from current position (all directions hit a boundary or wall).")
    return player_character.x, player_character.y # Player stays in current position

# Function to print a gravestone ASCII art
def print_gravestone(name="hero", death_txt="loser"):
    print("       ____")
    print("     /       \\")
    print("    |   RIP  |")
    print("    |   IN   |")
    print("    |  PEACE |")
    print("    |_______ |")
    print(" Here lies " + name + " " + death_txt)

# New function to handle chest room interaction (copied from previous step)
def handle_chest_room(current_x, current_y, current_room, game_should_quit, move_player_func, player_character, my_tower):
    """
    Handles interaction when player enters a 'C' (Chest) room.

    Args:
        current_x (int): The current row coordinate of the player.
        current_y (int): The current column coordinate of the player.
        current_room (Room):
        grid_size (int): The size of one dimension of the square grid.
        game_should_quit (bool): Flag to indicate if the game should quit.
        move_player_func (function): The move_player function to use for movement.
        player_character (Character): The player's character object with inventory.

    Returns:
        tuple: (updated_x, updated_y, game_should_quit, interacted_this_turn)
    """
    interacted_this_turn = False
    print("You are in a 'C' room. A chilling draft sweeps through.")
    print("A chest is here!")

    while True:
        chest_action = input("Enter 'o' to open the chest, 'i' to view items, a direction (n, s, e, w) to move, or 'q' to quit: ").lower()

        if chest_action == 'o':
            items = ['gold', 'gas', 'boom', 'empty', 'treasure']
            # Probabilities must sum to 1.
            # gold (65%), gas (10%), explosion (10%), empty (10%), treasure (5%)
            probabilities = [0.65, 0.10, 0.10, 0.10, 0.05]
            # Select a single item
            selected_item_np = random.choices(items, weights=probabilities, k=1)[0]
            if selected_item_np == 'gold':
                 # Scale gold found by tower level based on rolling a 4-sided die for each floor
                 gold_found = 0
                 for _ in range(player_character.z + 1):
                     gold_found += random.randint(1, 4)
                 gold_found *= 1 # Multiply as needed
                 print(f"You found {gold_found} gold!")
                 player_character.gold += gold_found
                 print(f"Player now has {player_character.gold} gold.")
            elif selected_item_np == 'gas':
                 print("Gas explodes from the chest! You stagger from the room!")
                 # Directly call move_player_randomly here
                 current_x, current_y = move_player_randomly(player_character, my_tower, move_player_func)
            elif selected_item_np == 'boom':
                 print("The chest explodes.")
                 boom_dmg = 0
                 for _ in range(player_character.z + 1):
                     boom_dmg += random.randint(1, 4)
                 player_character.take_damage_no_def(boom_dmg)
                 if not player_character.is_alive():
                        print_gravestone() # Print gravestone here
                        print(f"{player_character.name} has been defeated! Game Over.")
                        game_should_quit = True
                        interacted_this_turn = True
                        break # Exit combat loop and main game loop
            elif selected_item_np == 'empty':
                 print("The chest is empty.")
            elif selected_item_np == 'treasure':
                print("You found a sparkling treasure!")
                # Select a random treasure item from the templates
                if TREASURE_TEMPLATES:
                    chosen_treasure_template = random.choice(TREASURE_TEMPLATES)
                    # Create a new instance of the Treasure object
                    new_treasure = Treasure(name=chosen_treasure_template.name,
                                            description=chosen_treasure_template.description,
                                            gold_value=chosen_treasure_template.gold_value,
                                            value=chosen_treasure_template.value,
                                            benefit=chosen_treasure_template.benefit,
                                            level=chosen_treasure_template.level)
                    player_character.inventory.add_item(new_treasure)
                    new_treasure.collect(player_character) # Automatically collect the gold from the treasure
                else:
                    print("You found a treasure, but there are no defined treasure items!")

            current_room.room_type = '.' # Change room type to empty after opening
            interacted_this_turn = True
            break
        elif chest_action == 'i':
            # Handle inventory actions, this is where a turn might be consumed
            turn_taken_in_inventory = handle_inventory_menu(player_character, my_tower, my_tower.floors[player_character.z])
            if turn_taken_in_inventory:
                interacted_this_turn = True
                break # Exit interaction after taking a turn in inventory
            # If no turn taken (e.g., just viewed inventory), stay in loop to allow other chest actions
        elif chest_action == 'q':
            print("Exiting game.")
            game_should_quit = True
            interacted_this_turn = True # Set to true to break from outer loop
            break
        elif chest_action in ['n', 's', 'e', 'w']:
            old_ch_x, old_ch_y = current_x, current_y
            new_ch_x, new_ch_y = move_player_func(player_character,my_tower, chest_action)
            if (new_ch_x, new_ch_y) == (old_ch_x, old_ch_y):
                print("Cannot move further in that direction (boundary reached).")
            current_x, current_y = new_ch_x, new_ch_y
            interacted_this_turn = True # Set to true to break from outer loop and re-render
            break
        else:
            print("Invalid input for chest interaction. Please use 'o', 'i', n, s, e, w, or q.")

    return current_x, current_y, game_should_quit, interacted_this_turn




# Global list of monster templates
MONSTER_TEMPLATES = [
    {
        'name': "Slime Mold",
        'health': 15,
        'attack': 5,
        'defense': 0,
        'level': 0,
        'flavor_text': "A gelatinous blob quivers menacingly.",
        'victory_text': "The slime mold shimmers and then dissolves into an inert puddle.",
        'elemental_weakness': ['Fire'],
        'elemental_strength': ['Water']
    },
    {
        'name': "Sewer Rat",
        'health': 10,
        'attack': 4,
        'defense': 0,
        'level': 0,
        'flavor_text': "A scurrying rat, unnaturally large.",
        'victory_text': "The rat squeals its last and goes still.",
        'elemental_weakness': ['Physical'],
        'elemental_strength': ['Darkness']
    },
    {
        'name': "Bat",
        'health': 12,
        'attack': 3,
        'defense': 0,
        'level': 0,
        'flavor_text': "A leathery-winged bat swoops down.",
        'victory_text': "The bat drops silently to the floor.",
        'elemental_weakness': ['Wind'],
        'elemental_strength': ['Darkness']
    },
    {
        'name': "Giant Centipede",
        'health': 15,
        'attack': 5,
        'defense': 0,
        'level': 0,
        'flavor_text': "A many-legged horror skitters past.",
        'victory_text': "The centipede's legs curl up, motionless.",
        'elemental_weakness': ['Physical'],
        'elemental_strength': ['Earth']
    },
    {
        'name': "Lichen",
        'health': 10,
        'attack': 2,
        'defense': 0,
        'level': 0,
        'flavor_text': "A slow and menacing living amalgam of fungus and plant",
        'victory_text': "The lichen lies motionless at your feet.",
        'elemental_weakness': ['Fire'],
        'elemental_strength': ['Earth']
    },

    {
        'name': "Goblin",
        'health': 30,
        'attack': 10,
        'defense': 2,
        'level': 1,
        'flavor_text': "A small, green-skinned goblin snarls at you.",
        'victory_text': "The goblin yelps and falls, its snarl replaced by a whimper.",
        'elemental_weakness': ['Fire'],
        'elemental_strength': ['Physical']
    },
    {
        'name': "Skeleton",
        'health': 25,
        'attack': 12,
        'defense': 3,
        'level': 1,
        'flavor_text': "Rattling bones form a reanimated skeleton.",
        'victory_text': "The skeleton clatters to pieces, its unholy animation ceased.",
        'elemental_weakness': ['Holy'],
        'elemental_strength': ['Darkness']
    },
    {
        'name': "Kobold",
        'health': 20,
        'attack': 8,
        'defense': 1,
        'level': 1,
        'flavor_text': "A small, dog-like humanoid yaps angrily.",
        'victory_text': "The kobold whines as it falls, its primitive weapon clattering.",
        'elemental_weakness': ['Physical'],
        'elemental_strength': ['Fire']
    },
    {
        'name': "Giant Rat",
        'health': 22,
        'attack': 7,
        'defense': 1,
        'level': 1,
        'flavor_text': "A rat the size of a small dog, teeth bared.",
        'victory_text': "The giant rat twitches and is silent.",
        'elemental_weakness': ['Physical'],
        'elemental_strength': ['Darkness']
    },
    {
        'name': "Zombie",
        'health': 35,
        'attack': 9,
        'defense': 1,
        'level': 1,
        'flavor_text': "A shuffling corpse groans its approach.",
        'victory_text': "The zombie finally collapses into a heap of rotting flesh.",
        'elemental_weakness': ['Fire'],
        'elemental_strength': ['Darkness']
    },

    {
        'name': "Orc",
        'health': 50,
        'attack': 15,
        'defense': 5,
        'level': 2,
        'flavor_text': "A hulking orc with a crude axe blocks your path.",
        'victory_text': "The orc's roar dies in its throat as it collapses, its axe clattering to the ground.",
        'elemental_weakness': ['Physical'],
        'elemental_strength': ['Darkness']
    },
    {
        'name': "Giant Spider",
        'health': 40,
        'attack': 18,
        'defense': 4,
        'level': 2,
        'flavor_text': "A massive spider, its eyes glowing red, scuttles towards you.",
        'victory_text': "The giant spider shudders, its legs curl up, and it lies still.",
        'elemental_weakness': ['Fire'],
        'elemental_strength': ['Earth']
    },
    {
        'name': "Gnoll",
        'health': 45,
        'attack': 13,
        'defense': 3,
        'level': 2,
        'flavor_text': "A hyena-headed humanoid snarls, axe in hand.",
        'victory_text': "The gnoll's cackles fade into a gurgle.",
        'elemental_weakness': ['Physical'],
        'elemental_strength': ['Earth']
    },
    {
        'name': "Wight",
        'health': 40,
        'attack': 16,
        'defense': 4,
        'level': 2,
        'flavor_text': "A gaunt, shadowy figure drifts towards you.",
        'victory_text': "The wight dissipates into cold mist.",
        'elemental_weakness': ['Holy'],
        'elemental_strength': ['Darkness']
    },
    {
        'name': "Bugbear",
        'health': 55,
        'attack': 14,
        'defense': 4,
        'level': 2,
        'flavor_text': "A hairy, goblin-like brute lunges.",
        'victory_text': "The bugbear lets out a frustrated growl before falling.",
        'elemental_weakness': ['Physical'],
        'elemental_strength': ['Earth']
    },

    {
        'name': "Troll",
        'health': 70,
        'attack': 20,
        'defense': 6,
        'level': 3,
        'flavor_text': "A hulking troll grins, its wounds already closing.",
        'victory_text': "The troll collapses, its regeneration finally overcome.",
        'elemental_weakness': ['Fire'],
        'elemental_strength': ['Earth']
    },
    {
        'name': "Ogre",
        'health': 80,
        'attack': 18,
        'defense': 7,
        'level': 3,
        'flavor_text': "A brutish ogre wields a massive club.",
        'victory_text': "The ogre crashes down, its club rolling away.",
        'elemental_weakness': ['Physical'],
        'elemental_strength': ['Earth']
    },
    {
        'name': "Minotaur",
        'health': 75,
        'attack': 22,
        'defense': 6,
        'level': 3,
        'flavor_text': "A bull-headed monster snorts, ready to charge.",
        'victory_text': "The minotaur's roar turns to a death rattle.",
        'elemental_weakness': ['Physical'],
        'elemental_strength': ['Earth']
    },
    {
        'name': "Gargoyle",
        'health': 60,
        'attack': 25,
        'defense': 8,
        'level': 3,
        'flavor_text': "A stone gargoyle detaches from the wall, eyes glowing.",
        'victory_text': "The gargoyle shatters into lifeless stone fragments.",
        'elemental_weakness': ['Earth'],
        'elemental_strength': ['Wind']
    },
    {
        'name': "Hell Hound",
        'health': 65,
        'attack': 23,
        'defense': 5,
        'level': 3,
        'flavor_text': "A fiendish hound with eyes of flame bays menacingly.",
        'victory_text': "The hell hound whines once, and its flames die.",
        'elemental_weakness': ['Water'],
        'elemental_strength': ['Fire']
    },

    {
        'name': "Wraith",
        'health': 80,
        'attack': 28,
        'defense': 7,
        'level': 4,
        'flavor_text': "A spectral figure wails, draining life force.",
        'victory_text': "The wraith dissipates into an ethereal mist, its torment ended.",
        'elemental_weakness': ['Holy'],
        'elemental_strength': ['Darkness']
    },
    {
        'name': "Basilisk",
        'health': 90,
        'attack': 25,
        'defense': 10,
        'level': 4,
        'flavor_text': "A scaly lizard-like creature with a petrifying gaze.",
        'victory_text': "The basilisk's eyes glaze over, its petrifying gaze now harmless.",
        'elemental_weakness': ['Physical'],
        'elemental_strength': ['Earth']
    },
    {
        'name': "Specter",
        'health': 70,
        'attack': 26,
        'defense': 6,
        'level': 4,
        'flavor_text': "A ghostly apparition phases through the walls, seeking warmth.",
        'victory_text': "The specter lets out a final, mournful cry before fading away.",
        'elemental_weakness': ['Holy'],
        'elemental_strength': ['Darkness']
    },
    {
        'name': "Otyugh",
        'health': 110,
        'attack': 20,
        'defense': 12,
        'level': 4,
        'flavor_text': "A grotesque, tentacled beast emerges from the refuse.",
        'victory_text': "The otyugh collapses into the filth, its horrid form still.",
        'elemental_weakness': ['Physical'],
        'elemental_strength': ['Earth']
    },
    {
        'name': "Hill Giant",
        'health': 130,
        'attack': 22,
        'defense': 8,
        'level': 4,
        'flavor_text': "A lumbering giant with a massive club grunts menacingly.",
        'victory_text': "The hill giant crashes to the ground, shaking the very foundations.",
        'elemental_weakness': ['Physical'],
        'elemental_strength': ['Earth']
    },

    {
        'name': "Mummy",
        'health': 95,
        'attack': 30,
        'defense': 10,
        'level': 5,
        'flavor_text': "A desiccated corpse, bound in ancient linens, groans.",
        'victory_text': "The mummy unravels into dust, its ancient curse broken.",
        'elemental_weakness': ['Fire'],
        'elemental_strength': ['Earth']
    },
    {
        'name': "Grick",
        'health': 80,
        'attack': 28,
        'defense': 15,
        'level': 5,
        'flavor_text': "A worm-like predator with razor-sharp beaks and tentacles.",
        'victory_text': "The grick writhes, then falls limp, its strange form inert.",
        'elemental_weakness': ['Physical'],
        'elemental_strength': ['Earth']
    },
    {
        'name': "Grell",
        'health': 75,
        'attack': 32,
        'defense': 10,
        'level': 5,
        'flavor_text': "A floating brain with a beak and dangling tentacles.",
        'victory_text': "The grell's tentacles go slack, its psionic hum silenced.",
        'elemental_weakness': ['Psionic'],
        'elemental_strength': ['Wind']
    },
    {
        'name': "Umber Hulk",
        'health': 140,
        'attack': 25,
        'defense': 18,
        'level': 5,
        'flavor_text': "A massive insectoid with powerful claws and a confusing gaze.",
        'victory_text': "The umber hulk collapses, its chitinous shell cracking.",
        'elemental_weakness': ['Physical'],
        'elemental_strength': ['Earth']
    },
    {
        'name': "Displacer Beast",
        'health': 105,
        'attack': 35,
        'defense': 10,
        'level': 5,
        'flavor_text': "A six-legged panther-like creature with tentacles, appearing to be slightly out of phase.",
        'victory_text': "The displacer beast shimmers, then fades away into true nothingness.",
        'elemental_weakness': ['Light'],
        'elemental_strength': ['Darkness']
    },

    {
        'name': "Gorgon",
        'health': 160,
        'attack': 35,
        'defense': 15,
        'level': 6,
        'flavor_text': "A fearsome bull-like creature that can turn foes to stone.",
        'victory_text': "The gorgon's stony gaze loses its power as it crumbles.",
        'elemental_weakness': ['Demonic'],
        'elemental_strength': ['Earth']
    },
    {
        'name': "Iron Golem",
        'health': 180,
        'attack': 30,
        'defense': 25,
        'level': 6,
        'flavor_text': "A colossal golem forged of iron, nearly impervious to harm.",
        'victory_text': "The iron golem's movements cease, its metallic shell now just a statue.",
        'elemental_weakness': ['Earth'],
        'elemental_strength': ['Physical']
    },
    {
        'name': "Medusa",
        'health': 95,
        'attack': 48,
        'defense': 9,
        'level': 6,
        'flavor_text': "A serpentine woman whose gaze can turn you to stone.",
        'victory_text': "The medusa's stone-cold eyes lose their power, her serpentine form collapsing.",
        'elemental_weakness': ['Light'],
        'elemental_strength': ['Darkness']
    },
    {
        'name': "Lich",
        'health': 130,
        'attack': 45,
        'defense': 10,
        'level': 6,
        'flavor_text': "A skeletal sorcerer, eyes glowing with malevolent magic.",
        'victory_text': "The lich's phylactery shatters, and its form crumbles to dust.",
        'elemental_weakness': ['Holy'],
        'elemental_strength': ['Darkness']
    },
    {
        'name': "Vampire",
        'health': 110,
        'attack': 40,
        'defense': 10,
        'level': 6,
        'flavor_text': "A charismatic creature of the night, with a thirst for blood.",
        'victory_text': "The vampire hisses as it turns to dust, its reign of terror ended.",
        'elemental_weakness': ['Holy'],
        'elemental_strength': ['Darkness']
    },

    {
        'name': "Chimera",
        'health': 90,
        'attack': 30,
        'defense': 10,
        'level': 7,
        'flavor_text': "A terrifying beast with the heads of a lion, goat, and dragon.",
        'victory_text': "The chimera's multiple heads fall limp, its monstrous form still.",
        'elemental_weakness': ['Fire'],
        'elemental_strength': ['Earth']
    },
    {
        'name': "Beholder",
        'health': 100,
        'attack': 35,
        'defense': 12,
        'level': 7,
        'flavor_text': "A floating orb with a single central eye and multiple eyestalks.",
        'victory_text': "The beholder's eyes go dark, its antimagic cone dissipating.",
        'elemental_weakness': ['Psionic'],
        'elemental_strength': ['Light']
    },
    {
        'name': "Young Dragon",
        'health': 120,
        'attack': 32,
        'defense': 15,
        'level': 7,
        'flavor_text': "A scaled beast, not yet fully grown, snorts smoke.",
        'victory_text': "The young dragon lets out a final, weak roar.",
        'elemental_weakness': ['Ice'],
        'elemental_strength': ['Fire']
    },
    {
        'name': "Stone Golem",
        'health': 150,
        'attack': 28,
        'defense': 20,
        'level': 7,
        'flavor_text': "A massive construct of animated stone lumbers forward.",
        'victory_text': "The stone golem crumbles into an inert pile of rocks.",
        'elemental_weakness': ['Earth'],
        'elemental_strength': ['Physical']
    },
    {
        'name': "Mind Flayer",
        'health': 85,
        'attack': 40,
        'defense': 8,
        'level': 7,
        'flavor_text': "A humanoid with a tentacled face probes your thoughts.",
        'victory_text': "The mind flayer's tentacles writhe before it collapses, its psionic presence gone.",
        'elemental_weakness': ['Psionic'],
        'elemental_strength': ['Darkness']
    },

    {
        'name': "Giant Frost Worm",
        'health': 170,
        'attack': 38,
        'defense': 15,
        'level': 8,
        'flavor_text': "A colossal worm of ice and teeth bursts from the ground.",
        'victory_text': "The giant frost worm shudders, then freezes solid and cracks apart.",
        'elemental_weakness': ['Fire'],
        'elemental_strength': ['Ice']
    },
    {
        'name': "Fire Elemental",
        'health': 100,
        'attack': 45,
        'defense': 10,
        'level': 8,
        'flavor_text': "A swirling vortex of living flame, eager to immolate.",
        'victory_text': "The fire elemental's flames die down, leaving only smoke and ash.",
        'elemental_weakness': ['Water'],
        'elemental_strength': ['Fire']
    },
    {
        'name': "Griffin",
        'health': 110,
        'attack': 30,
        'defense': 12,
        'level': 8,
        'flavor_text': "A noble beast with the head of an eagle and body of a lion, it screeches a challenge.",
        'victory_text': "The griffin cries out once, then collapses, its majestic form now still.",
        'elemental_weakness': ['Wind'],
        'elemental_strength': ['Physical']
    },
    {
        'name': "Naga Guardian",
        'health': 120,
        'attack': 33,
        'defense': 13,
        'level': 8,
        'flavor_text': "A serpentine humanoid with ancient eyes, guarding forgotten treasures.",
        'victory_text': "The naga hisses its last, its coils unwrapping in defeat.",
        'elemental_weakness': ['Holy'],
        'elemental_strength': ['Water']
    },
    {
        'name': "Roc",
        'health': 190,
        'attack': 30,
        'defense': 18,
        'level': 8,
        'flavor_text': "A monstrous bird of prey, casting a shadow as it descends.",
        'victory_text': "The roc tumbles from the sky, a thunderous crash marking its end.",
        'elemental_weakness': ['Wind'],
        'elemental_strength': ['Earth']
    },

    {
        'name': "Sphinx",
        'health': 150,
        'attack': 40,
        'defense': 20,
        'level': 9,
        'flavor_text': "A creature of wisdom and riddles, it awaits your answer.",
        'victory_text': "The sphinx lets out a soft sigh as it fades into ancient lore.",
        'elemental_weakness': ['Psionic'],
        'elemental_strength': ['Light']
    },
    {
        'name': "Purple Worm",
        'health': 200,
        'attack': 45,
        'defense': 20,
        'level': 9,
        'flavor_text': "A gargantuan worm, its maw capable of swallowing a horse whole.",
        'victory_text': "The purple worm thrashes violently, then goes limp, its tunneling days over.",
        'elemental_weakness': ['Earth'],
        'elemental_strength': ['Physical']
    },
    {
        'name': "Efreeti",
        'health': 140,
        'attack': 48,
        'defense': 15,
        'level': 9,
        'flavor_text': "A powerful genie of fire, with a temper as hot as its domain.",
        'victory_text': "The efreeti's fiery form shrinks, leaving behind only a scorch mark.",
        'elemental_weakness': ['Water'],
        'elemental_strength': ['Fire']
    },
    {
        'name': "Cyclops",
        'health': 220,
        'attack': 35,
        'defense': 25,
        'level': 9,
        'flavor_text': "A one-eyed giant, its simple mind focused on crushing intruders.",
        'victory_text': "The cyclops roars in pain, then falls, shaking the very earth.",
        'elemental_weakness': ['Physical'],
        'elemental_strength': ['Earth']
    },
    {
        'name': "Dragon Turtle",
        'health': 180,
        'attack': 50,
        'defense': 22,
        'level': 9,
        'flavor_text': "A massive amphibious dragon, its shell a fortress, its breath steam.",
        'victory_text': "The dragon turtle retreats into the depths, its scales scarred and broken.",
        'elemental_weakness': ['Ice'],
        'elemental_strength': ['Water']
    },

    {
        'name': "Demilich ",
        'health': 180,
        'attack': 55,
        'defense': 30,
        'level': 10,
        'flavor_text': "A floating skull, jewels glinting in its empty eye sockets, laughs maniacally.",
        'victory_text': "The demilich's mocking laughter turns to a silent crumble.",
        'elemental_weakness': ['Holy'],
        'elemental_strength': ['Darkness']
    },
    {
        'name': "Ancient Dragon",
        'health': 200,
        'attack': 50,
        'defense': 25,
        'level': 10,
        'flavor_text': "An immense, ancient dragon fills the cavern with its presence.",
        'victory_text': "The ancient dragon lets out a deafening roar as it falls, shaking the very foundations of the tower.",
        'elemental_weakness': ['Ice'],
        'elemental_strength': ['Fire']
    },
    {
        'name': "Balrog",
        'health': 160,
        'attack': 48,
        'defense': 22,
        'level': 10,
        'flavor_text': "A creature of shadow and flame, wielding a fiery whip and sword.",
        'victory_text': "The balrog's fire dims, and its shadow form dissipates.",
        'elemental_weakness': ['Holy'],
        'elemental_strength': ['Fire']
    },
    {
        'name': "Elder Brain",
        'health': 140,
        'attack': 42,
        'defense': 18,
        'level': 10,
        'flavor_text': "A massive, pulsating brain, psionically controlling its surroundings.",
        'victory_text': "The elder brain's psychic screams cease, and its form goes inert.",
        'elemental_weakness': ['Psionic'],
        'elemental_strength': ['Darkness']
    },
    {
        'name': "Tarrasque",
        'health': 250,
        'attack': 60,
        'defense': 35,
        'level': 10,
        'flavor_text': "A legendary, indestructible creature of immense size and power, here to devour all.",
        'victory_text': "The Tarrasque's gargantuan form finally collapses, its reign of terror over as the world breathes a sigh of relief.",
        'elemental_weakness': ['Demonic'],
        'elemental_strength': ['Physical']
    }
]

# Sort the monster templates by level
MONSTER_TEMPLATES.sort(key=lambda x: x['level'])

# Define item templates for various types
WEAPON_TEMPLATES = [
    Weapon(name="Dagger", description="A small, sharp blade.", attack_bonus=2, value=10, level=0, upgrade_level=0),
    Weapon(name="Short Sword", description="A balanced one-handed sword.", attack_bonus=4, value=25, level=0, upgrade_level=0),
    Weapon(name="Iron Sword", description="A sturdy iron sword.", attack_bonus=5, value=75, level=1, upgrade_level=0),
    Weapon(name="Battle Axe", description="A heavy axe for close combat.", attack_bonus=7, value=120, level=2, upgrade_level=0),
    Weapon(name="Longsword", description="A classic adventurer's sword.", attack_bonus=6, value=100, level=2, upgrade_level=0),
    Weapon(name="Greatsword", description="A massive two-handed sword.", attack_bonus=10, value=250, level=3, upgrade_level=0),
    Weapon(name="Warhammer", description="A crushing blow with this hammer.", attack_bonus=8, value=180, level=3, upgrade_level=0),
    #Weapon(name="Bow", description="A simple wooden bow.", attack_bonus=3, value=30, level=0, upgrade_level=0),
    Weapon(name="Staff", description="A simple wooden staff, good for bludgeoning.", attack_bonus=3, value=15, level=0, upgrade_level=0, upgrade_limit=False)
]

ARMOR_TEMPLATES = [
    Armor(name="Leather Armor", description="Light leather armor.", defense_bonus=3, value=50, level=0, upgrade_level=0),
    Armor(name="Chainmail", description="Interlocking metal rings.", defense_bonus=5, value=90, level=3, upgrade_level=0),
    Armor(name="Plate Armor", description="Heavy, protective metal plates.", defense_bonus=8, value=150, level=5, upgrade_level=0),
    Armor(name="Studded Leather", description="Leather reinforced with metal studs.", defense_bonus=4, value=70, level=2, upgrade_level=0),
    Armor(name="Scale Mail", description="Overlapping scales of metal.", defense_bonus=6, value=110, level=4, upgrade_level=0)
]

POTION_TEMPLATES = [
    Potion(name="Minor Healing Potion", description="Restores a small amount of health.", healing_amount=15, value=15, level=0),
    Potion(name="Healing Potion", description="Restores a moderate amount of health.", healing_amount=30, value=25, level=1),
    Potion(name="Greater Healing Potion", description="Restores a large amount of health.", healing_amount=50, value=40, level=2),
    Potion(name="Strength Potion", description="Boosts strength.", potion_type='stat_boost', stat_to_boost='strength', boost_amount=2, value=30, level=1),
    Potion(name="Dexterity Potion", description="Boosts dexterity.", potion_type='stat_boost', stat_to_boost='dexterity', boost_amount=2, value=30, level=1),
    Potion(name="Intelligence Potion", description="Boosts intelligence.", potion_type='stat_boost', stat_to_boost='intelligence', boost_amount=2, value=30, level=1),
    Potion(name="Experience Potion", description="Grants a burst of experience.", potion_type='experience', experience_gain=50, value=500, level=2)
]

SCROLL_TEMPLATES = [
    Scroll(name="Scroll of Minor Protection", description="A scroll that grants minor magical protection.", effect_description="Temporarily increases defense.", value=20, level=0),
    Scroll(name="Scroll of Foresight", description="A scroll that reveals what lies ahead.", effect_description="Reveals nearby rooms.", value=40, level=1),
    Scroll(name="Scroll of Fireball", description="A scroll containing a powerful fire spell.", effect_description="Unleashes a fireball.", value=70, level=2),
    Scroll(name="Scroll of Upgrade", description="A mystical scroll that can enhance a weapon or armor.", effect_description="Permanently increases an item's upgrade level.", value=100, level=2, scroll_type='upgrade')
]

UTILITY_TEMPLATES = [
    Flare(name="Flare", description="Emits bright light for a short duration.", duration_rounds=3, light_radius=5, value=15, level=0),
    Lantern(name="Lantern", description="Provides continuous light with fuel.", fuel_amount=10, light_radius=7, value=30, level=0)
]

TREASURE_TEMPLATES = [
    Treasure(name="Small Gem", description="A tiny, sparkling gem.", gold_value=20, value=20, benefit={}, level=0),
    Treasure(name="Silver Necklace", description="A delicate silver chain.", gold_value=40, value=40, benefit={}, level=1),
    Treasure(name="Gold Coin Pouch", description="A heavy pouch filled with gold coins.", gold_value=100, value=100, benefit={}, level=1),
    Treasure(name="Ornate Goblet", description="A beautifully crafted goblet.", gold_value=75, value=75, benefit={}, level=1),
    Treasure(name="Amulet of Health", description="An amulet that slightly increases health.", gold_value=60, value=60, benefit={'health': 5}, level=2),
    Treasure(name="Ring of Protection", description="A ring that offers minor protection.", gold_value=80, value=80, benefit={'defense': 1}, level=2)
]

ALL_ITEM_TEMPLATES = (
    WEAPON_TEMPLATES +
    ARMOR_TEMPLATES +
    POTION_TEMPLATES +
    SCROLL_TEMPLATES +
    UTILITY_TEMPLATES +
    TREASURE_TEMPLATES
)

UNIQUE_WEAPON_TEMPLATES = [
    Weapon(name="Eldrich Sword", description="A wicked dark blade radiating magic.", attack_bonus=14, value=1500, level=20, upgrade_level=4, upgrade_limit=False, elemental_strength='Darkness'),
    Weapon(name="Blade of the Phoenix", description="A sword that burns with eternal flame, said to rise from ashes.", attack_bonus=18, value=2500, level=25, upgrade_level=5, upgrade_limit=False, elemental_strength='Fire'),
    Weapon(name="Stormbreaker Axe", description="A mighty axe that calls upon the fury of storms, capable of splitting mountains.", attack_bonus=20, value=2800, level=30, upgrade_level=6, upgrade_limit=False, elemental_strength='Wind'),
    Weapon(name="Scepter of the Ancients", description="A staff imbued with primordial magic, granting wisdom and devastating arcane power.", attack_bonus=12, value=2000, level=22, upgrade_level=7, upgrade_limit=False, elemental_strength='Psionic'),
    Weapon(name="Whisperwind Bow", description="An elven bow that shoots arrows silent as the wind, striking foes with unerring accuracy.", attack_bonus=16, value=2300, level=28, upgrade_level=5, upgrade_limit=False, elemental_strength='Wind'),
    Weapon(name="Dragon's Tooth Blade", description="A sword forged from a dragon's fang, glowing with draconic energy.", attack_bonus=25, value=3500, level=35, upgrade_level=7, upgrade_limit=False, elemental_strength='Fire'),
    Weapon(name="Hammer of the Titans", description="A colossal hammer wielded by ancient giants, capable of shattering mountains.", attack_bonus=30, value=4000, level=40, upgrade_level=8, upgrade_limit=False, elemental_strength='Earth'),
    Weapon(name="Soulreaver Scythe", description="A sinister scythe that hungers for souls, said to steal life with every strike.", attack_bonus=28, value=3800, level=38, upgrade_level=6, upgrade_limit=False, elemental_strength='Darkness'),
    Weapon(name="Shieldbreaker Maul", description="A heavy maul designed to crush defenses, leaving enemies vulnerable.", attack_bonus=22, value=3000, level=32, upgrade_level=7, upgrade_limit=False, elemental_strength='Physical'),
    Weapon(name="Vorpal Blade", description="A legendary sword with an insatiable thirst for blood, known for its decapitating strikes.", attack_bonus=32, value=4500, level=42, upgrade_level=9, upgrade_limit=False, elemental_strength='Physical')
]

UNIQUE_ARMOR_TEMPLATES = [
    Armor(name="Eldrich Mail", description="An imposing suit of armor.", defense_bonus=12, value=2300, level=20, upgrade_level=8, upgrade_limit=False, elemental_strength='Darkness'),
    Armor(name="Dragon Scale Armor", description="Forged from the scales of a mighty dragon, offering supreme protection.", defense_bonus=15, value=3000, level=25, upgrade_level=9, upgrade_limit=False, elemental_strength='Fire'),
    Armor(name="Aegis of the Paladin", description="A holy plate armor imbued with divine protection, radiating a calming aura.", defense_bonus=18, value=3800, level=30, upgrade_level=10, upgrade_limit=False, elemental_strength='Holy'),
    Armor(name="Shadow Cloak of Nocturne", description="A cloak woven from pure shadow, granting the wearer unparalleled stealth and defense against dark magic.", defense_bonus=10, value=2500, level=22, upgrade_level=8, upgrade_limit=False, elemental_strength='Darkness'),
    Armor(name="Titan's Plate", description="Massive, heavy plate armor said to have been worn by a Titan, offering incredible physical defense.", defense_bonus=20, value=4200, level=35, upgrade_level=12, upgrade_limit=False, elemental_strength='Earth'),
    Armor(name="Runesmith's Carapace", description="Dwarven plate armor etched with ancient runes, deflecting both physical and magical attacks.", defense_bonus=17, value=3500, level=28, upgrade_level=11, upgrade_limit=False, elemental_strength='Physical')
]

# Sort ALL_ITEM_TEMPLATES by item level
ALL_ITEM_TEMPLATES.sort(key=lambda item: item.level)

def handle_altar_room(player_character, current_room, game_should_quit, move_player_func, my_tower):
    """
    Handles interaction when player enters an 'A' (Altar) room.
    Allows the player to pray to different gods with random outcomes.

    Args:
        player_character (Character): The player's character object.
        current_room (Room): The Room object the player is currently in.
        game_should_quit (bool): Flag to indicate if the game should quit.
        grid_size (int): The size of one dimension of the square grid.
        move_player_func (function): The move_player function to use for movement.

    Returns:
        tuple: (updated_x, updated_y, game_should_quit, interacted_this_turn)
    """
    interacted_this_turn = False
    print("You are in an 'A' room. An ancient altar stands before you, radiating mystical energy.")

    gods = {
        1: {'name': 'The Benevolent One', 'type': 'good', 'blessing': 'You feel a surge of vitality.', 'effect': 'health', 'amount': 25},
        2: {'name': 'The Swift Hand', 'type': 'good', 'blessing': 'Your movements feel lighter.', 'effect': 'dexterity', 'amount': 2},
        3: {'name': 'The Wise Eye', 'type': 'good', 'blessing': 'Your mind sharpens.', 'effect': 'intelligence', 'amount': 2},
        4: {'name': 'The Strong Arm', 'type': 'good', 'blessing': 'You feel mightier.', 'effect': 'strength', 'amount': 2},
        5: {'name': 'The Trickster', 'type': 'neutral', 'blessing': 'You sense a mischievous presence.', 'effect': 'random', 'chance_good': 0.7, 'amount': 1},
        6: {'name': 'The Empty Silence', 'type': 'neutral', 'blessing': 'The altar remains silent.', 'effect': 'none', 'amount': 0},
        7: {'name': 'The Shadow Lord', 'type': 'evil', 'blessing': 'A chill runs down your spine.', 'effect': 'health', 'amount': -15}
    }

    # 1. Select a 'blessed' god for this altar encounter (must be a 'good' god)
    good_god_nums = [num for num, info in gods.items() if info['type'] == 'good']
    blessed_god_num = random.choice(good_god_nums)
    blessed_god_info = gods[blessed_god_num]

    # 2. Determine if the player gets a hunch based on intelligence
    # Base chance of 10% + 1% per intelligence point
    hunch_chance = 0.10 + (player_character.intelligence * 0.01)
    player_got_hunch = random.random() < hunch_chance
    hunch_revealed_god_num = None

    if player_got_hunch:
        # If player gets a hunch, it's about the blessed god
        print(f"\n GOOD hunch chance: {hunch_chance}")
        hunch_revealed_god_num = blessed_god_num
        print(f"\nYour inner wisdom whispers a name... you have a strong hunch about {blessed_god_info['name']}!")
    else:
        # If no hunch, maybe a misleading or no specific hunch.
        # For simplicity, let's just say no strong hunch if player_got_hunch is False.
        false_god_num = random.choice(good_god_nums)
        false_god_info = gods[false_god_num]
        print(f"\n FAIL hunch chance: {hunch_chance}")
        print(f"\nYour inner wisdom whispers a name... you have a strong hunch about {false_god_info['name']}!")


    while True:
        print("\nWhich deity will you pray to?")
        for num, god_info in gods.items():
            #hunch_indicator = " (You have a hunch!)" if num == hunch_revealed_god_num else ""
            print(f"  {num}. {god_info['name']} ({god_info['type'].capitalize()})")

        action = input("Enter a number to pray, 'i' to view items, a direction (n, s, e, w) to move, or 'q' to quit: ").lower()

        if action.isdigit() and int(action) in gods:
            chosen_god_num = int(action)
            chosen_god = gods[chosen_god_num]

            is_enhanced_blessing = (chosen_god_num == blessed_god_num)
            if is_enhanced_blessing:
                print(f"You pray to {chosen_god['name']}... {chosen_god['blessing']}")

                if chosen_god['effect'] == 'random':
                    # For 'The Trickster', if a hunch was correct, force a good outcome and enhance it.
                    if random.random() < chosen_god['chance_good']:
                        stats_to_boost = ['strength', 'dexterity', 'intelligence']
                        chosen_stat = random.choice(stats_to_boost)
                        amount = chosen_god['amount']
                        setattr(player_character, chosen_stat, getattr(player_character, chosen_stat) + amount)
                        print(f"Your {chosen_stat} increased by {amount}. Current {chosen_stat}: {getattr(player_character, chosen_stat)})")
                    else:
                        stats_to_decrease = ['strength', 'dexterity', 'intelligence']
                        chosen_stat = random.choice(stats_to_decrease)
                        amount = chosen_god['amount']
                        current_stat_value = getattr(player_character, chosen_stat)
                        setattr(player_character, chosen_stat, max(1, current_stat_value - amount))
                        print(f"Your {chosen_stat} decreased by {amount}. Current {chosen_stat}: {getattr(player_character, chosen_stat)})")
                elif chosen_god['effect'] == 'none':
                    print("Nothing seems to happen.")
                elif chosen_god['effect'] == 'health':
                    amount = chosen_god['amount']
                    if amount > 0:
                        player_character.health = player_character.health + amount
                        print(f"Your health increased by {amount}. Current health: {player_character.health})")
                    else:
                        player_is_defeated = player_character.take_damage_no_def(abs(amount))
                        if player_is_defeated:
                            print_gravestone(player_character.name, "was cursed")
                            print(f"{player_character.name} has been defeated! Game Over.")
                            game_should_quit = True
                elif chosen_god['effect'] == 'gold':
                    amount = chosen_god['amount']
                    player_character.gold += amount
                    print(f"You gained {amount} gold. Total gold: {player_character.gold}")
                else: # For other stats like strength, dexterity, intelligence
                    stat_name = chosen_god['effect']
                    amount = chosen_god['amount']
                    current_stat_value = getattr(player_character, stat_name)
                    setattr(player_character, stat_name, max(1, current_stat_value + amount))
                    print(f"Your {stat_name} changed by {amount}. Current {stat_name}: {getattr(player_character, stat_name)})")
            else:
              print(f"You pray to {chosen_god['name']}... {blessed_god_info['name']} smites you for your foolishness!")
              amount = blessed_god_info['amount'] * (player_character.z+1)
              player_is_defeated = player_character.take_damage_no_def(abs(amount))
              if player_is_defeated:
                 print_gravestone(player_character.name, "was smited")
                 print(f"{player_character.name} has been defeated! Game Over.")
                 game_should_quit = True
            print(f"The alter to {blessed_god_info['name']} crumbles to dust.")
            current_room.room_type = '.' # Altar is now used
            interacted_this_turn = True
            break # Exit interaction loop after praying

        elif action == 'i':
            turn_taken_in_inventory = handle_inventory_menu(player_character, my_tower)
            if turn_taken_in_inventory:
                interacted_this_turn = True
            # Stay in loop to allow other altar actions if no item was used

        elif action == 'q':
            print("Exiting game.")
            game_should_quit = True
            interacted_this_turn = True
            break

        elif action in ['n', 's', 'e', 'w']:
            old_ch_x, old_ch_y = player_character.x, player_character.y
            new_ch_x, new_ch_y = move_player_func(player_character,my_tower, action)
            if (new_ch_x, new_ch_y) == (old_ch_x, old_ch_y):
                print("Cannot move further in that direction (boundary reached).")
            player_character.x, player_character.y = new_ch_x, new_ch_y
            interacted_this_turn = True
            break # Exit interaction loop after moving

        else:
            print("Invalid input for altar interaction. Please use a number, 'i', n, s, e, w, or q.")

        if game_should_quit:
            break

    return player_character.x, player_character.y, game_should_quit, interacted_this_turn


# New function to handle monster room interaction
def handle_monster_room(current_x, current_y, current_room, game_should_quit, move_player_func, player_character, encountered_monsters, my_tower):
    """
    Handles interaction when player enters an 'M' (Monster) room.

    Args:
        current_x (int): The current row coordinate of the player.
        current_y (int): The current column coordinate of the player.
        current_room (Room): The Room object the player is currently in.
        grid_size (int): The size of one dimension of the square grid.
        game_should_quit (bool): Flag to indicate if the game should quit.
        move_player_func (function): The move_player function to use for movement.
        player_character (Character): The player's character object with inventory.
        encountered_monsters (dict): Dictionary to store persistent monsters.

    Returns:
        tuple: (updated_x, updated_y, game_should_quit, interacted_this_turn) - interacted_this_turn is True if player took an action, False otherwise
    """
    interacted_this_turn = False
    print("You are in a 'M' room. A mysterious mist fills the air.")

    monster_coords = (player_character.x, player_character.y, player_character.z)
    if monster_coords not in encountered_monsters:
        # Calculate effective monster level based on tower floor
        # Each 'level' of monster difficulty corresponds to roughly 5 floors
        floor_monster_level = player_character.z // 5

        # Filter monster templates to find suitable monsters
        # Monsters should be around the floor's effective level, possibly slightly higher
        suitable_monsters = [m for m in MONSTER_TEMPLATES if m['level'] <= floor_monster_level + 1]

        # If no suitable monsters are found (e.g., very early floors with no level 0 monsters, or max level reached)
        if not suitable_monsters:
            # If it's a very low floor and no level 0 monster, pick the absolute lowest level monster.
            if floor_monster_level == 0 and [m for m in MONSTER_TEMPLATES if m['level'] == 0]:
                suitable_monsters = [m for m in MONSTER_TEMPLATES if m['level'] == 0]
            else:
                # Otherwise, default to the highest level monster available if past all designed levels
                max_level_available = max(m['level'] for m in MONSTER_TEMPLATES)
                suitable_monsters = [m for m in MONSTER_TEMPLATES if m['level'] == max_level_available]

        # Select a random monster from the suitable ones
        template = random.choice(suitable_monsters)

        # 2. Create a new Monster object using the template and store it
        current_monster = Monster(name=template['name'], health=template['health'], attack=template['attack'], defense=template['defense'], level=template['level'], elemental_strength=template['elemental_strength'], elemental_weakness=template['elemental_weakness'])
        encountered_monsters[monster_coords] = current_monster
        print(f"A new {current_monster.name} appears! {template['flavor_text']}")
    else:
        # Use the previously encountered monster
        current_monster = encountered_monsters[monster_coords]
        print(f"The {current_monster.name} you encountered earlier is still here!")

    # 3. Print a message announcing the monster (flavor text can be added here or in the template)
    print(f"A {current_monster.name} stands before you!\n")

    # Combat loop
    while True:
        # Calculate monster's hit chance for the current round
        base_monster_hit_chance = 0.8
        # Adjustments:
        # - Increased by monster's level (e.g., +2% per level)
        # - Decreased by player's level (e.g., -1% per level)
        # - Decreased by player's dexterity (e.g., -0.5% per dexterity point)
        monster_hit_chance = max(0, min(1, base_monster_hit_chance +
                             (current_monster.level * 0.02) -
                             (player_character.level * 0.01) -
                             (player_character.dexterity * 0.005)))

        print(f"{player_character.name} HP: {player_character.health} | {current_monster.name} HP: {current_monster.health})")
        monster_action = input("Enter 'a' to attack, 'f' to flee, 'i' to view items, or 'q' to quit: ").lower()

        if monster_action == 'a':
            # Player's turn
            # need to change hit probability based on level
            if random.random() < 0.7: # 70% chance to hit
                player_damage = max(1, player_character.attack_target(current_monster) - current_monster.defense)
                print(f"{player_character.name} hits {current_monster.name} for {player_damage} damage!")
                monster_defeated = current_monster.take_damage(player_damage)

                if monster_defeated:
                    print(f"You defeated the {current_monster.name}!")
                    # Print victory flavor text
                    print(template['victory_text'])
                    gained_gold = (current_monster.level+1)*(player_character.z+1)*random.randint(1,6)
                    gained_exp = (current_monster.level+1)*(player_character.z+1)*5
                    player_character.gold += gained_gold
                    player_character.experience += gained_exp
                    print(f"You gained {gained_gold} gold and {gained_exp} experience. Total gold: {player_character.gold}, Total exp: {player_character.experience}")
                    #add check to next level exp
                    #add current level to ui
                    current_room.room_type = '.' # Change room type to empty after monster is defeated
                    del encountered_monsters[monster_coords] # Remove defeated monster
                    interacted_this_turn = True
                    break # Exit combat loop
            else:
                print(f"{player_character.name} missed {current_monster.name}!")

            # Monster's turn (if still alive)
            if current_monster.is_alive():
                if random.random() < monster_hit_chance: # Use calculated hit chance
                    current_monster.attack_target(player_character)
                    if not player_character.is_alive():
                        print_gravestone() # Print gravestone here
                        print(f"{player_character.name} has been defeated! Game Over.")
                        game_should_quit = True
                        interacted_this_turn = True
                        break # Exit combat loop and main game loop
                else:
                    print(f"{current_monster.name} missed {player_character.name}!")

        elif monster_action == 'f':
            flee_success_chance = 0.6 # 60% chance to flee successfully
            if random.random() < flee_success_chance:
                print("You successfully fled!")
                while True:
                    direction_choice = input("Choose a direction to escape (n, s, e, w): ").lower()
                    if direction_choice in ['n', 's', 'e', 'w']:
                        old_ch_x, old_ch_y = current_x, current_y
                        new_ch_x, new_ch_y = move_player_func(player_character,my_tower, direction_choice)
                        if (new_ch_x, new_ch_y) == (old_ch_x, old_ch_y):
                            print("Cannot move further in that direction (boundary reached).")
                        else:
                            current_x, current_y = new_ch_x, new_ch_y
                            interacted_this_turn = True
                            break # Break from inner direction choice loop
                    else:
                        print("Invalid direction. Please choose n, s, e, or w.")
                break # Break from outer monster interaction loop
            else:
                print("You failed to flee! The monster blocks your path!")
                # Monster attacks if flee failed
                if random.random() < monster_hit_chance: # Use calculated hit chance
                    current_monster.attack_target(player_character)
                    if not player_character.is_alive():
                        print_gravestone() # Print gravestone here
                        print(f"{player_character.name} has been defeated! Game Over.")
                        game_should_quit = True
                        interacted_this_turn = True
                        break # Exit combat loop and main game loop
                else:
                    print(f"{current_monster.name} missed {player_character.name}!")

        elif monster_action == 'i':
            # Handle inventory actions, this is where a turn might be consumed
            turn_taken_in_inventory = handle_inventory_menu(player_character, my_tower, in_combat=True)
            if turn_taken_in_inventory:
                interacted_this_turn = True # An action in inventory counts as a turn
                # After using an item, monster still gets a turn if alive and game not quit
                if current_monster.is_alive() and not game_should_quit:
                    if random.random() < monster_hit_chance:
                        current_monster.attack_target(player_character)
                        if not player_character.is_alive():
                            print_gravestone() # Print gravestone here
                            print(f"{player_character.name} has been defeated! Game Over.")
                            game_should_quit = True
                            interacted_this_turn = True
                            break # Exit combat loop and main game loop
                    else:
                        print(f"{current_monster.name} missed {player_character.name}!")

        elif monster_action == 'q':
            print("Exiting game.")
            game_should_quit = True
            interacted_this_turn = True
            break

        else:
            print("Invalid input for monster interaction. Please use 'f', 'r', 'i', or q.")

        # Check if game should quit after any action (fight, flee, use item, quit)
        if game_should_quit:
            break

    return current_x, current_y, game_should_quit, interacted_this_turn

# New function to handle vendor room interaction
def handle_vendor_room(current_x, current_y, current_room, game_should_quit, move_player_func, player_character, vendor_character, my_tower):
    """
    Handles interaction when player enters a 'V' (Vendor) room.

    Args:
        current_x (int): The current row coordinate of the player.
        current_y (int): The current column coordinate of the player.
        current_room (Room): The Room object the player is currently in.
        grid_size (int): The size of one dimension of the square grid.
        game_should_quit (bool): Flag to indicate if the game should quit.
        move_player_func (function): The move_player function to use for movement.
        player_character (Character): The player's character object with inventory.
        vendor_character (Character): The vendor's character object with inventory and gold.

    Returns:
        tuple: (updated_x, updated_y, game_should_quit, interacted_this_turn)
    """
    interacted_this_turn = False
    game_should_quit = False
    print("You are in a 'V' room. A friendly vendor greets you.")

    while True:
        print(f"\nYour gold: {player_character.gold}")
        print("Vendor's wares:")
        if not vendor_character.inventory.items:
            print("  The vendor has no items left to sell.")
        else:
            for i, item in enumerate(vendor_character.inventory.items):
                item_display = f"  {i + 1}. {item.name} (Value: {item.value} gold)"
                if isinstance(item, (Weapon, Armor)):
                    item_display += f" +{item.upgrade_level}"
                print(item_display)

        print("\nYour inventory:")
        player_character.inventory.display_inventory()
        print(f"Equipped Weapon: {player_character.equipped_weapon.name if player_character.equipped_weapon else 'None'}")
        print(f"Equipped Armor: {player_character.equipped_armor.name if player_character.equipped_armor else 'None'}")

        vendor_action = input("Enter 'b [number]' to buy, 's [number]' to sell, 'f' to finish shopping, or 'q' to quit: ").lower()

        if vendor_action.startswith('b '):
            try:
                item_number_to_buy = int(vendor_action[2:].strip()) - 1
                if 0 <= item_number_to_buy < len(vendor_character.inventory.items):
                    item_to_buy = vendor_character.inventory.items[item_number_to_buy]
                    if player_character.gold >= item_to_buy.value:
                        player_character.gold -= item_to_buy.value
                        vendor_character.gold += item_to_buy.value
                        # Create a copy of the item for the player's inventory
                        # This prevents the vendor's item object from being directly moved, important if vendor has multiple copies
                        if isinstance(item_to_buy, Weapon):
                            new_item = Weapon(item_to_buy.name, item_to_buy.description, item_to_buy._base_attack_bonus, item_to_buy.value, item_to_buy.level, item_to_buy.upgrade_level)
                        elif isinstance(item_to_buy, Armor):
                            new_item = Armor(item_to_buy.name, item_to_buy.description, item_to_buy._base_defense_bonus, item_to_buy.value, item_to_buy.level, item_to_buy.upgrade_level)
                        elif isinstance(item_to_buy, Potion):
                            new_item = Potion(item_to_buy.name, item_to_buy.description, item_to_buy.healing_amount, item_to_buy.potion_type, item_to_buy.stat_to_boost, item_to_buy.boost_amount, item_to_buy.experience_gain, item_to_buy.value, item_to_buy.level)
                        elif isinstance(item_to_buy, Scroll):
                            new_item = Scroll(item_to_buy.name, item_to_buy.description, item_to_buy.effect_description, item_to_buy.value, item_to_buy.level, item_to_buy.scroll_type)
                        elif isinstance(item_to_buy, Flare):
                            new_item = Flare(item_to_buy.name, item_to_buy.description, item_to_buy.duration_rounds, item_to_buy.light_radius, item_to_buy.value, item_to_buy.level)
                        elif isinstance(item_to_buy, Lantern):
                            new_item = Lantern(item_to_buy.name, item_to_buy.description, item_to_buy.fuel_amount, item_to_buy.light_radius, item_to_buy.value, item_to_buy.level)
                        elif isinstance(item_to_buy, Treasure):
                            new_item = Treasure(name=item_to_buy.name, description=item_to_buy.description, gold_value=item_to_buy.gold_value, value=item_to_buy.value, benefit=item_to_buy.benefit, level=item_to_buy.level)
                        else:
                            new_item = Item(item_to_buy.name, item_to_buy.description, item_to_buy.value, item_to_buy.level)

                        player_character.inventory.add_item(new_item)
                        vendor_character.inventory.items.pop(item_number_to_buy) # Directly remove by index
                        print(f"You bought {new_item.name} for {new_item.value} gold. You now have {player_character.gold} gold.")
                        interacted_this_turn = True
                    else:
                        print(f"You don't have enough gold to buy {item_to_buy.name}. You need {item_to_buy.value} gold.")
                else:
                    print("Invalid item number.")
            except ValueError:
                print("Invalid input. Please enter 'b' followed by the item number.")

        elif vendor_action.startswith('s '):
            try:
                item_number_to_sell = int(vendor_action[2:].strip()) - 1
                if 0 <= item_number_to_sell < len(player_character.inventory.items):
                    item_to_sell = player_character.inventory.items[item_number_to_sell]
                    # For simplicity, vendor buys at half price
                    sell_price = item_to_sell.value // 2
                    if vendor_character.gold >= sell_price:
                        player_character.gold += sell_price
                        vendor_character.gold -= sell_price
                        # Vendor gets a copy of the item. This creates a new object so player's original is distinct.
                        if isinstance(item_to_sell, Weapon):
                            new_item_for_vendor = Weapon(item_to_sell.name, item_to_sell.description, item_to_sell._base_attack_bonus, item_to_sell.value, item_to_sell.level, item_to_sell.upgrade_level)
                        elif isinstance(item_to_sell, Armor):
                            new_item_for_vendor = Armor(item_to_sell.name, item_to_sell.description, item_to_sell._base_defense_bonus, item_to_sell.value, item_to_sell.level, item_to_sell.upgrade_level)
                        elif isinstance(item_to_sell, Potion):
                            new_item_for_vendor = Potion(item_to_sell.name, item_to_sell.description, item_to_sell.healing_amount, item_to_sell.potion_type, item_to_sell.stat_to_boost, item_to_sell.boost_amount, item_to_sell.experience_gain, item_to_sell.value, item_to_sell.level)
                        elif isinstance(item_to_sell, Scroll):
                            new_item_for_vendor = Scroll(item_to_sell.name, item_to_sell.description, item_to_sell.effect_description, item_to_sell.value, item_to_sell.level, item_to_sell.scroll_type)
                        elif isinstance(item_to_sell, Flare):
                            new_item_for_vendor = Flare(item_to_sell.name, item_to_sell.description, item_to_sell.duration_rounds, item_to_sell.light_radius, item_to_sell.value, item_to_sell.level)
                        elif isinstance(item_to_sell, Lantern):
                            new_item_for_vendor = Lantern(item_to_sell.name, item_to_sell.description, item_to_sell.fuel_amount, item_to_sell.light_radius, item_to_sell.value, item_to_sell.level)
                        elif isinstance(item_to_sell, Treasure):
                            new_item_for_vendor = Treasure(name=item_to_sell.name, description=item_to_sell.description, gold_value=item_to_sell.gold_value, value=item_to_sell.value, benefit=item_to_sell.benefit, level=item_to_sell.level)
                        else:
                            new_item_for_vendor = Item(item_to_sell.name, item_to_sell.description, item_to_sell.value, item_to_sell.level)

                        vendor_character.inventory.add_item(new_item_for_vendor) # Vendor gets the item
                        player_character.inventory.remove_item(item_to_sell.name) # Player no longer has it
                        print(f"You sold {item_to_sell.name} for {sell_price} gold. You now have {player_character.gold} gold.")
                        interacted_this_turn = True
                    else:
                        print(f"The vendor doesn't have enough gold to buy {item_to_sell.name}.")
                else:
                    print("Invalid item number.")
            except ValueError:
                print("Invalid input. Please enter 's' followed by the item number.")
        elif vendor_action == 'f':
            print("You finish shopping and prepare for your adventure!")
            break
        elif vendor_action == 'q':
            print("Exiting game.")
            game_should_quit = True
            interacted_this_turn = True
            break
        else:
            print("Invalid input. Please use 'b [number]', 's [number]', 'f', or 'q'.")
    return current_x, current_y, game_should_quit, interacted_this_turn

vendor_names = ["Bartholomew", "Agnes", "Old Man Tiberius", "Madame Xylos", "Flimsy Fred"]

def create_player_character(ch_x, ch_y):
    print("\n--- Character Creation ---")
    player_name = input("Enter your hero's name: ")

    # Define available classes, races, and genders with base stats/modifiers
    character_races = {
        'human': {'health_mod': 0, 'attack_mod': 0, 'defense_mod': 0, 'strength_mod': 0, 'dexterity_mod': 0, 'intelligence_mod': 0},
        'elf': {'health_mod': -10, 'attack_mod': 1, 'defense_mod': -1, 'strength_mod': -1, 'dexterity_mod': 2, 'intelligence_mod': 2},
        'dwarf': {'health_mod': 20, 'attack_mod': 2, 'defense_mod': 2, 'strength_mod': 2, 'dexterity_mod': -2, 'intelligence_mod': -2}
    }

    character_genders = {
        'male': {'attack_mod': 0, 'defense_mod': 0},
        'female': {'attack_mod': 0, 'defense_mod': 0},
        'non-binary': {'attack_mod': 0, 'defense_mod': 0}
    }

    # Base stats for the 'Hero' class (since there's only one class now)
    base_stats = {'health': 100, 'attack': 15, 'defense': 5, 'strength': 10, 'dexterity': 10, 'intelligence': 10}

    # Choose race
    while True:
        print("Choose your race: (H for Human, E for Elf, D for Dwarf)")
        race_choice = input("> ").lower()
        if race_choice == 'h':
            chosen_race = 'human'
            break
        elif race_choice == 'e':
            chosen_race = 'elf'
            break
        elif race_choice == 'd':
            chosen_race = 'dwarf'
            break
        else:
            print("Invalid race choice. Please enter H, E, or D.")

    # Choose gender
    while True:
        print("Choose your gender: (M for Male, F for Female, N for Non-binary)")
        gender_choice = input("> ").lower()
        if gender_choice == 'm':
            chosen_gender = 'male'
            break
        elif gender_choice == 'f':
            chosen_gender = 'female'
            break
        elif gender_choice == 'n':
            chosen_gender = 'non-binary'
            break
        else:
            print("Invalid gender choice. Please enter M, F, or N.")

    # Apply base stats
    stats = base_stats.copy()

    # Apply race modifiers
    race_mods = character_races[chosen_race]
    stats['health'] += race_mods.get('health_mod', 0)
    stats['attack'] += race_mods.get('attack_mod', 0)
    stats['defense'] += race_mods.get('defense_mod', 0)
    stats['strength'] += race_mods.get('strength_mod', 0)
    stats['dexterity'] += race_mods.get('dexterity_mod', 0)
    stats['intelligence'] += race_mods.get('intelligence_mod', 0)

    # Apply gender modifiers
    gender_mods = character_genders[chosen_gender]
    stats['attack'] += gender_mods.get('attack_mod', 0)
    stats['defense'] += gender_mods.get('defense_mod', 0)

    # Create character object
    player = Character(
        name=player_name,
        health=stats['health'],
        attack=stats['attack'],
        defense=stats['defense'],
        strength=stats['strength'],
        dexterity=stats['dexterity'],
        intelligence=stats['intelligence'],
        x=ch_x, y=ch_y, z=0,
        character_class="Adventurer", # Default to Adventurer class
        race=chosen_race,
        gender=chosen_gender
    )

    print(f"\nWelcome, {player.name} the {player.gender} {player.race} {player.character_class}!")
    print(f"Health: {player.health}, Attack: {player.attack}, Defense: {player.defense}, Strength: {player.strength}, Dexterity: {player.dexterity}, Intelligence: {player.intelligence}")
    return player

def generate_random_vendor(player_level, player_character):
    """
    Generates a random vendor with a randomized inventory scaled to player level.

    Args:
        player_level (int): The current level of the player character, used to scale item values.

    Returns:
        Character: A Character object representing the vendor.
    """
    vendor_name = random.choice(vendor_names)

    # Scale vendor gold based on player level
    vendor_gold = 200 + (player_level * 50) + random.randint(-50, 50)
    if vendor_gold < 0: vendor_gold = 0

    vendor = Character(name=vendor_name, health=100, attack=0, defense=0, strength=5, dexterity=5, intelligence=10, level=1, gold=vendor_gold)

    # Define a pool of possible items to draw from, weighted by type popularity or usefulness
    # For simplicity, we'll give each type a list and pick from those
    item_type_pools = [
        [item for item in WEAPON_TEMPLATES if item.level <= player_level + 1],
        [item for item in ARMOR_TEMPLATES if item.level <= player_level + 1],
        [item for item in POTION_TEMPLATES if item.level <= player_level + 1],
        [item for item in SCROLL_TEMPLATES if item.level <= player_level + 1]
        #[item for item in UTILITY_TEMPLATES if item.level <= player_level + 1]
        #[item for item in TREASURE_TEMPLATES if item.level <= player_level + 1]
    ]
    # Flatten the list of lists into a single pool for random choices
    all_available_items = [item for sublist in item_type_pools for item in sublist]

    # Randomly select a number of items for the vendor, e.g., 3 to 6 items
    num_items = random.randint(3, 6)
    vendor_wares = random.sample(all_available_items, num_items) if all_available_items else []

    #player_character.inventory.check_item()

    #add required utliltiy items to vendor wares
    item_type_req = [
        [item for item in UTILITY_TEMPLATES if item.level <= player_level + 1]
    ]
    all_required_items = [item for sublist in item_type_req for item in sublist]
    vendor_wares += all_required_items

    for item in vendor_wares:
        # Scale item value for the vendor slightly, e.g., +/- 10% based on player level
        scaled_value = item.value + (item.value * (random.uniform(-0.1, 0.1) + (player_level * 0.01)))
        item.value = max(5, int(scaled_value)) # Ensure value is at least 5
        # Add a copy of the item to the vendor's inventory, ensuring it's a distinct object
        if isinstance(item, Weapon):
            vendor.inventory.add_item_quiet(Weapon(item.name, item.description, item._base_attack_bonus, item.value, item.level, item.upgrade_level))
        elif isinstance(item, Armor):
            vendor.inventory.add_item_quiet(Armor(item.name, item.description, item._base_defense_bonus, item.value, item.level, item.upgrade_level))
        elif isinstance(item, Potion):
            vendor.inventory.add_item_quiet(Potion(item.name, item.description, item.healing_amount, item.potion_type, item.stat_to_boost, item.boost_amount, item.experience_gain, item.value, item.level))
        elif isinstance(item, Scroll):
            vendor.inventory.add_item_quiet(Scroll(item.name, item.description, item.effect_description, item.value, item.level, item.scroll_type))
        elif isinstance(item, Flare):
            vendor.inventory.add_item_quiet(Flare(item.name, item.description, item.duration_rounds, item.light_radius, item.value, item.level))
        elif isinstance(item, Lantern):
            vendor.inventory.add_item_quiet(Lantern(item.name, item.description, item.fuel_amount, item.light_radius, item.value, item.level))
        elif isinstance(item, Treasure):
            vendor.inventory.add_item_quiet(Treasure(name=item.name, description=item.description, gold_value=item.gold_value, value=item.value, benefit=item.benefit, level=item.level))
        else:
            vendor.inventory.add_item_quiet(Item(item.name, item.description, item.value, item.level))

    return vendor

def handle_stairs_up(player_character, my_tower, floor_params):
    interacted_this_turn = False
    print("You found stairs leading up!")
    player_character.z += 1

    if player_character.z >= len(my_tower.floors):
        # Generate a new floor if it doesn't exist yet
        my_tower.add_floor(**floor_params)
        print(f"Generated new Floor {player_character.z + 1}!")
    else:
        print(f"Ascending to Floor {player_character.z + 1}!")

    current_floor = my_tower.floors[player_character.z]
    # Find the 'D' room on the new floor to place the character
    found_d = False
    for r_idx, row in enumerate(current_floor.grid):
            for c_idx, room in enumerate(row):
                if room.room_type == 'D': # Now looking for 'D' for descending stairs
                    player_character.x = r_idx
                    player_character.y = c_idx
                    found_d = True
                    break
            if found_d:
                break
    interacted_this_turn = True
    print(f"You arrive at ({player_character.x}, {player_character.y}) on Floor {player_character.z + 1}.")
    return player_character.x, player_character.y, False, interacted_this_turn # False for game_should_quit

def handle_stairs_down(player_character, my_tower, move_player_func):
    interacted_this_turn = False
    print("You found stairs leading down!")

    if player_character.z == 0:
        print("You are on the ground floor (Tower Entry). Are you sure you want to exit the tower?")
        exit_choice = input("Type 'yes' to exit, or any other key to stay, or a direction (n,s,e,w) to move, or 'q' to quit: ").lower()
        if exit_choice == 'yes':
            print("You exit the tower. Game Over.")
            return player_character.x, player_character.y, True, True # True for game_should_quit, True for interacted
        elif exit_choice in ['n', 's', 'e', 'w']:
            old_ch_x, old_ch_y = player_character.x, player_character.y
            new_ch_x, new_ch_y = move_player_func(player_character,my_tower, exit_choice) # Fixed NameError: current_x, current_y -> player_character.x, player_character.y
            if (new_ch_x, new_ch_y) == (old_ch_x, old_ch_y):
                print("Cannot move further in that direction (boundary reached).")
            player_character.x, player_character.y = new_ch_x, new_ch_y
            interacted_this_turn = True
            return player_character.x, player_character.y, False, interacted_this_turn # Explicit return to prevent None
        else:
            print("You decide not to exit the tower.")
            return player_character.x, player_character.y, False, False # Stay in current room, not interacted
    else:
        player_character.z -= 1
        print(f"Descending to Floor {player_character.z + 1}!")

        current_floor = my_tower.floors[player_character.z]
        # Find the 'U' room on the previous floor to place the character
        found_u = False
        for r_idx, row in enumerate(current_floor.grid):
            for c_idx, room in enumerate(row):
                if room.room_type == 'U': # Now looking for 'U' for ascending stairs
                    player_character.x = r_idx
                    player_character.y = c_idx
                    found_u = True
                    break
            if found_u:
                break
    interacted_this_turn = True
    print(f"You arrive at ({player_character.x}, {player_character.y}) on Floor {player_character.z + 1}.")
    return player_character.x, player_character.y, False, interacted_this_turn # False for game_should_quit

def handle_inventory_menu(player_character, my_tower, in_combat=False):
    """
    Handles player interactions within the inventory menu.
    Allows viewing, using, equipping, and unequipping items.

    Args:
        player_character (Character): The player's character object.
        my_tower (Tower): The Tower object.
        grid_size (int): The size of one dimension of the square grid.
        in_combat (bool): True if the player is currently in combat, which might restrict some actions.

    Returns:
        bool: True if an item was used or equipped/unequipped, signifying a turn taken.
              False if the player just viewed inventory or went back without taking an action.
    """
    turn_taken = False
    while True:
        print("\n--- Inventory ---")
        player_character.inventory.display_inventory()
        print(f"Gold: {player_character.gold}")
        print(f"Equipped Weapon: {player_character.equipped_weapon.name if player_character.equipped_weapon else 'None'}")
        print(f"Equipped Armor: {player_character.equipped_armor.name if player_character.equipped_armor else 'None'}")
        print("-----------------")

        action = input("Enter 'u [item #]' to use, 'e [item #]' to equip, 'x' to unequip (w/a), 'b' to go back: ").lower()

        if action.startswith('u '):
            try:
                item_index = int(action[2:].strip()) - 1
                if 0 <= item_index < len(player_character.inventory.items):
                    item_obj = player_character.inventory.items[item_index]
                    used_successfully = False
                    if isinstance(item_obj, (Flare, Lantern)):
                        used_successfully = item_obj.use(player_character, my_tower)
                    else:
                        used_successfully = item_obj.use(player_character)

                    if used_successfully: # use() returns True if consumed/actioned
                        player_character.inventory.remove_item(item_obj.name)
                        turn_taken = True
                        break # Exit inventory after using an item
                else:
                    print("Invalid item number.")
            except ValueError:
                print("Invalid input. Please enter 'u' followed by the item number.")

        elif action.startswith('e '):
            try:
                item_index = int(action[2:].strip()) - 1
                if 0 <= item_index < len(player_character.inventory.items):
                    item_obj = player_character.inventory.items[item_index]
                    if isinstance(item_obj, (Weapon, Armor)):
                        player_character.equip_item(item_obj)
                        turn_taken = True
                        break # Exit inventory after equipping
                    else:
                        print("You can only equip weapons or armor.")
                else:
                    print("Invalid item number.")
            except ValueError:
                print("Invalid input. Please enter 'e' followed by the item number.")

        elif action == 'x': # Unequip
            unequip_choice = input("Unequip 'w' for weapon or 'a' for armor? ").lower()
            if unequip_choice == 'w':
                player_character.unequip_weapon()
                turn_taken = True
                break
            elif unequip_choice == 'a':
                player_character.unequip_armor()
                turn_taken = True
                break
            else:
                print("Invalid choice. Please enter 'w' or 'a'.")

        elif action == 'b':
            break # Exit inventory menu
        else:
            print("Invalid inventory action.")
    return turn_taken

def handle_pool_room(current_x, current_y, current_room, game_should_quit, move_player_func, player_character, my_tower):
    """
    Handles interaction when player enters a 'P' (Pool) room.
    Allows the player to drink water from a basin with random outcomes.

    Args:
        current_x (int): The current row coordinate of the player.
        current_y (int): The current column coordinate of the player.
        current_room (Room): The Room object the player is currently in.
        grid_size (int): The size of one dimension of the square grid.
        game_should_quit (bool): Flag to indicate if the game should quit.
        move_player_func (function): The move_player function to use for movement.
        player_character (Character): The player's character object.

    Returns:
        tuple: (updated_x, updated_y, game_should_quit, interacted_this_turn)
    """
    interacted_this_turn = False
    print("You are in a 'P' room. A basin of glowing water shimmers before you.")

    while True:
        action = input("Enter 'd' to drink, 'i' to view items, a direction (n, s, e, w) to move, or 'q' to quit: ").lower()

        if action == 'd':
            outcomes = ['stat_increase', 'stat_decrease', 'poison']
            chosen_outcome = random.choice(outcomes)

            if chosen_outcome == 'stat_increase':
                stats_to_boost = ['health', 'strength', 'dexterity', 'intelligence']
                chosen_stat = random.choice(stats_to_boost)

                if chosen_stat == 'health':
                    amount = random.randint(15, 25)
                    player_character.health = player_character.health + amount
                    print(f"You feel invigorated! Your health increased by {amount}. Current health: {player_character.health})")
                else:
                    amount = random.randint(1, 3)
                    setattr(player_character, chosen_stat, getattr(player_character, chosen_stat) + amount)
                    print(f"You feel a surge of energy! Your {chosen_stat} increased by {amount}. Current {chosen_stat}: {getattr(player_character, chosen_stat)})")

            elif chosen_outcome == 'stat_decrease':
                stats_to_decrease = ['strength', 'dexterity', 'intelligence']
                chosen_stat = random.choice(stats_to_decrease)
                amount = random.randint(1, 2)

                current_stat_value = getattr(player_character, chosen_stat)
                setattr(player_character, chosen_stat, max(1, current_stat_value - amount)) # Ensure stat doesn't go below 1
                print(f"You feel a strange weakness! Your {chosen_stat} decreased by {amount}. Current {chosen_stat}: {getattr(player_character, chosen_stat)})")

            elif chosen_outcome == 'poison':
                damage = random.randint(10, 20)
                print("The water tastes foul! You've been poisoned!")
                player_is_defeated = player_character.take_damage_no_def(damage)

                if player_is_defeated:
                    print_gravestone(player_character.name, "was poisoned")
                    print(f"{player_character.name} has been defeated! Game Over.")
                    game_should_quit = True

            current_room.room_type = '.' # Basin is now empty
            interacted_this_turn = True
            break # Exit interaction loop after drinking

        elif action == 'i':
            turn_taken_in_inventory = handle_inventory_menu(player_character, my_tower)
            if turn_taken_in_inventory:
                interacted_this_turn = True
                break # Exit interaction after taking a turn in inventory

        elif action == 'q':
            print("Exiting game.")
            game_should_quit = True
            interacted_this_turn = True
            break

        elif action in ['n', 's', 'e', 'w']:
            old_ch_x, old_ch_y = current_x, current_y
            new_ch_x, new_ch_y = move_player_func(player_character,my_tower, action)
            if (new_ch_x, new_ch_y) == (old_ch_x, old_ch_y):
                print("Cannot move further in that direction (boundary reached).")
            current_x, current_y = new_ch_x, new_ch_y
            interacted_this_turn = True
            break

        else:
            print("Invalid input for pool interaction. Please use 'd', 'i', n, s, e, w, or q.")

        if game_should_quit:
            break

    return current_x, current_y, game_should_quit, interacted_this_turn

def handle_warp_room(current_x, current_y, current_room, game_should_quit, my_tower, player_character, floor_params):
    """
    Handles interaction when player enters a 'W' (Warp) room.

    Args:
        current_x (int): The current row coordinate of the player.
        current_y (int): The current column coordinate of the player.
        current_room (Room): The Room object the player is currently in.
        grid_size (int): The size of one dimension of the square grid.
        game_should_quit (bool): Flag to indicate if the game should quit.
        my_tower (Tower): The Tower object.
        player_character (Character): The player's character object.
        specified_chars (list): List of specified room characters.
        required_chars (list): List of required room characters.

    Returns:
        tuple: (updated_x, updated_y, game_should_quit, interacted_this_turn)
    """
    interacted_this_turn = False
    print("You are in a 'W' room. A swirling portal shimmers before you!")

    # Calculate evasion chance based on intelligence
    evasion_chance = 0.20 + (player_character.intelligence * 0.02) # Base 20% + 2% per intelligence point
    evasion_chance = min(1.0, evasion_chance) # Cap at 100%

    if random.random() < evasion_chance:
        print(f"Your keen intellect allows you to resist the portal's pull! You manage to avoid being warped.")
        # No movement or turn consumed specific to the warp, player can move normally.
        # This means the game loop will ask for input again.
    else:
        print("The portal's power overwhelms you! You are sucked in...")

        # Determine new floor (within +/- 3 floors)
        min_z = max(0, player_character.z - 3)
        max_z = player_character.z + 3

        # Ensure max_z does not exceed newly generated floor (if any) to prevent errors
        # This might require generating floors up to max_z if they don't exist.
        while len(my_tower.floors) <= max_z:
            # When generating new floors due to warp, use the standard required_chars including 'D', 'U', 'V'
            my_tower.add_floor(**floor_params)

        new_z = random.randint(min_z, max_z)
        player_character.z = new_z

        # Determine new (x,y) on the new floor
        #work to do here
        #check if not wall
        while True:
          new_x = random.randint(0, my_tower.floors[player_character.z].rows - 1)
          new_y = random.randint(0, my_tower.floors[player_character.z].cols - 1)
          if is_wall_at_coordinate(my_tower.floors[player_character.z], new_x, new_y):
            continue
          else:
            break

        player_character.x = new_x
        player_character.y = new_y

        print(f"You emerge disoriented in a new room: ({player_character.x}, {player_character.y}) on Floor {player_character.z + 1}!")
        interacted_this_turn = True # A turn was definitely consumed by warping

    return player_character.x, player_character.y, game_should_quit, interacted_this_turn

def handle_starting_shop(player_character, my_tower):
    print("\n--- Welcome to the Starting Shop! ---")
    print("Here you can purchase essential gear before venturing into the tower.")
    starting_vendor = generate_random_vendor(player_character.level, player_character) # Generate a vendor for the shop
    print(f"Today's merchant is {starting_vendor.name}.")

    # The handle_vendor_room function now returns a tuple, so we need to unpack it.
    # However, for the starting shop, we only care about game_should_quit.
    # The player's position won't change, and interacted_this_turn isn't used for the initial shop flow.
    _, _, game_quit_from_shop, _ = handle_vendor_room(0, 0, None, False, move_player, player_character, starting_vendor, my_tower)
    return game_quit_from_shop

#MAIN GAME

# Define the specified characters
specified_chars = ['M', 'C', 'A', 'P', 'W', '.']

# Characters that must appear exactly once
required_chars = ['D', 'U', 'V']

# Define desired floor dimensions and characters (from cell_b81U3DaaCMlJ)
grid_rows = 15
grid_cols = 15
wall_char = '#'
floor_char = '.'

# Character limits (example values, will be used in populate_rooms later)
p_limits_val = (2, 5)
c_limits_val = (1, 4)
w_limits_val = (3, 6)
a_limits_val = (2, 5)

floor_params={
    'specified_chars': specified_chars,
    'required_chars': required_chars,
    'grid_rows': grid_rows,
    'grid_cols': grid_cols,
    'wall_char': wall_char,
    'floor_char': floor_char,
    'p_limits': p_limits_val,
    'c_limits': c_limits_val,
    'w_limits': w_limits_val,
    'a_limits': a_limits_val
}

# Create an instance of the `Tower` class
my_tower = Tower()

# Call the `add_floor` method to generate the first floor
my_tower.add_floor(specified_chars, required_chars, grid_rows, grid_cols, wall_char, floor_char,
                   p_limits=p_limits_val, c_limits=c_limits_val, w_limits=w_limits_val, a_limits=a_limits_val)

# The player starts on floor 0.
# 'E' (entrance/exit) is now fixed at (1,1) on the first floor.
ch_x, ch_y = 1, 1

# Define grid_size for the move_player function
#grid_size = 5

# Instantiate player_character using the character creation dialog
player_character = create_player_character(ch_x, ch_y)
player_character.gold = 500 # Give starting gold

# No longer giving starting items directly, they can buy them
# player_character.inventory.add_item(Weapon(name="Dagger", description="A small, sharp blade.", attack_bonus=2, value=10, level=0, upgrade_level=0))
# player_character.inventory.add_item(Potion(name="Minor Healing Potion", description="Restores a small amount of health.", healing_amount=15, value=15, level=0))
# player_character.inventory.add_item(Scroll(name="Scroll of Upgrade", description="A mystical scroll that can enhance a weapon or armor.", effect_description="Permanently increases an item's upgrade level.", value=100, level=2, scroll_type='upgrade'))

# Initialize a boolean variable game_should_quit = False before the main while True game loop.
game_should_quit = False

# Handle initial shopping phase
game_should_quit = handle_starting_shop(player_character, my_tower)

# Store vendors encountered for each vendor room (x, y, z) -> vendor_character
encountered_vendors = {}
# Store monsters encountered for each monster room (x, y, z) -> monster_character
encountered_monsters = {}

# Game loop for player movement
while True:
    # Add another boolean variable interacted_with_special_room = False.
    interacted_with_special_room = False

    # Clear the console for a cleaner display, if running in a terminal-like environment
    # This might not get executed in all environments (e.g., some Jupyter/Colab setups)
    os.system('cls' if os.name == 'nt' else 'clear')

    current_floor = my_tower.floors[player_character.z]
    # Mark current room as discovered
    current_floor.grid[player_character.x][player_character.y].discovered = True

    #reveal walls
    reveal_cells_around_character(player_character, my_tower, radius=1)

    current_floor.print_floor(highlight_coords=(player_character.x, player_character.y))

        # Print player's current position and floor
    print(f"Current player position: ({player_character.x}, {player_character.y}) on Floor {player_character.z + 1}")

    # Monster regeneration: All encountered monsters heal 1 HP
    for monster_key in list(encountered_monsters.keys()): # Use list() to allow dictionary modification during iteration
        monster = encountered_monsters[monster_key]
        if monster.is_alive() and monster.health < monster.max_health:
            monster.health = min(monster.health + 1, monster.max_health)
            # print(f"{monster.name} regenerated 1 HP. Current HP: {monster.health}") # Optional: uncomment for verbose healing

    # Retrieve current room and its type
    current_room = current_floor.grid[player_character.x][player_character.y]
    current_room_type = current_room.room_type

    # Add conditional statements for room interactions
    if current_room_type == 'U':
        print("You are in a 'U' room. There's an unusual hum here.")
        action = input("Enter 'u' to ascend, 'i' to view items, or a direction (n,s,e,w) to move, or 'q' to quit: ").lower()
        if action == 'u':
            player_character.x, player_character.y, game_should_quit, interacted_with_special_room = \
                handle_stairs_up(player_character, my_tower, floor_params)
        elif action == 'i':
            turn_taken_in_inventory = handle_inventory_menu(player_character, my_tower)
            if turn_taken_in_inventory:
                interacted_with_special_room = True
        elif action == 'q':
            game_should_quit = True
            interacted_with_special_room = True
        elif action in ['n', 's', 'e', 'w']:
            old_ch_x, old_ch_y = player_character.x, player_character.y
            new_ch_x, new_ch_y = move_player(player_character, my_tower, action)
            if (new_ch_x, new_ch_y) == (old_ch_x, old_ch_y):
                print("Cannot move further in that direction (boundary reached).")
            player_character.x, player_character.y = new_ch_x, new_ch_y
            interacted_with_special_room = True
        else:
            print("Invalid input for 'U' room. Continuing in current room.")
            interacted_with_special_room = False
    elif current_room_type == 'E': # Now checking for 'E' for entrance/exit
        print("You are in the 'E' room. This is the tower's entrance/exit.")
        action = input("Enter 'x' to exit the tower, 'i' to view items, or a direction (n,s,e,w) to move, or 'q' to quit: ").lower()
        if action == 'x':
            player_character.x, player_character.y, game_should_quit, interacted_with_special_room = \
                handle_stairs_down(player_character, my_tower, move_player)
        elif action == 'i':
            turn_taken_in_inventory = handle_inventory_menu(player_character, my_tower)
            if turn_taken_in_inventory:
                interacted_with_special_room = True
        elif action == 'q':
            game_should_quit = True
            interacted_with_special_room = True
        elif action in ['n', 's', 'e', 'w']:
            old_ch_x, old_ch_y = player_character.x, player_character.y
            new_ch_x, new_ch_y = move_player(player_character, my_tower, action)
            if (new_ch_x, new_ch_y) == (old_ch_x, old_ch_y):
                print("Cannot move further in that direction (boundary reached).")
            player_character.x, player_character.y = new_ch_x, new_ch_y
            interacted_with_special_room = True
        else:
            print("Invalid input for 'E' room. Continuing in current room.")
            interacted_with_special_room = False
    elif current_room_type == 'D': # Now checking for 'D' stairs down
        print("You are in the 'D' room. This is stairs to a lower level.")
        action = input("Enter 'd' to descend, 'i' to view items, or a direction (n,s,e,w) to move, or 'q' to quit: ").lower()
        if action == 'd':
            player_character.x, player_character.y, game_should_quit, interacted_with_special_room = \
                handle_stairs_down(player_character, my_tower, move_player)
        elif action == 'i':
            turn_taken_in_inventory = handle_inventory_menu(player_character, my_tower)
            if turn_taken_in_inventory:
                interacted_with_special_room = True
        elif action == 'q':
            game_should_quit = True
            interacted_with_special_room = True
        elif action in ['n', 's', 'e', 'w']:
            old_ch_x, old_ch_y = player_character.x, player_character.y
            new_ch_x, new_ch_y = move_player(player_character, my_tower, action)
            if (new_ch_x, new_ch_y) == (old_ch_x, old_ch_y):
                print("Cannot move further in that direction (boundary reached).")
            player_character.x, player_character.y = new_ch_x, new_ch_y
            interacted_with_special_room = True
        else:
            print("Invalid input for 'D' room. Continuing in current room.")
            interacted_with_special_room = False
    elif current_room_type == 'M':
        # Call the new handle_monster_room function
        player_character.x, player_character.y, game_should_quit, interacted_with_special_room = handle_monster_room(player_character.x, player_character.y, current_room, game_should_quit, move_player, player_character, encountered_monsters, my_tower)
    elif current_room_type == 'C':
        # Call the new handle_chest_room function
        player_character.x, player_character.y, game_should_quit, interacted_with_special_room = handle_chest_room(player_character.x, player_character.y, current_room, game_should_quit, move_player, player_character, my_tower)
    elif current_room_type == 'A':
        player_character.x, player_character.y, game_should_quit, interacted_with_special_room = handle_altar_room(player_character, current_room, game_should_quit, move_player, my_tower)
    elif current_room_type == 'P':
        player_character.x, player_character.y, game_should_quit, interacted_with_special_room = handle_pool_room(player_character.x, player_character.y, current_room, game_should_quit, move_player, player_character, my_tower)
    elif current_room_type == 'W':
        player_character.x, player_character.y, game_should_quit, interacted_with_special_room = handle_warp_room(player_character.x, player_character.y, current_room, game_should_quit, my_tower, player_character, floor_params)
    elif current_room_type == 'V':
        vendor_coords = (player_character.x, player_character.y, player_character.z) # Vendor location now includes floor (z)
        if vendor_coords not in encountered_vendors:
            # Generate a new vendor if not encountered before
            new_vendor = generate_random_vendor(player_character.level, player_character)
            encountered_vendors[vendor_coords] = new_vendor
            print(f"You found a new vendor: {new_vendor.name}!")
        else:
            # Use the previously generated vendor
            new_vendor = encountered_vendors[vendor_coords]
            print(f"You returned to {new_vendor.name}'s shop.")

        player_character.x, player_character.y, game_should_quit, interacted_with_special_room = handle_vendor_room(player_character.x, player_character.y, current_room, game_should_quit, move_player, player_character, new_vendor, my_tower)
    elif current_room_type == '.':
        print("You are in an empty room. Nothing seems out of the ordinary.")

    reveal_cells_around_character(player_character, my_tower, radius=1)

    # Wrap the existing general movement input prompt and its associated logic inside an if not interacted_with_special_room: block.
    if not interacted_with_special_room:
        player_input = input("Enter direction (n, s, e, w), 'i' for inventory, or 'q' to quit: ").lower()

        if player_input == 'q':
            game_should_quit = True
            # No need to break here, the check after the if block will handle it
        elif player_input in ['n', 's', 'e', 'w']:
            new_ch_x, new_ch_y = move_player(player_character, my_tower, player_input)
            if (new_ch_x, new_ch_y) == (player_character.x, player_character.y):
                print("Cannot move further in that direction (boundary reached).")
            player_character.x, player_character.y = new_ch_x, new_ch_y
        elif player_input == 'i':
            turn_taken_in_inventory = handle_inventory_menu(player_character, my_tower)
            if turn_taken_in_inventory:
                interacted_with_special_room = True # If an item was used, it counts as an interaction
        else:
            print("Invalid input. Please use n, s, e, w, 'i', or q.")

    # Add a check to exit the main game loop if game_should_quit is True
    if game_should_quit:
       player_input = input("Are you sure you want to quit? (y/n)").lower()
       if player_input == 'y':
            print("Exiting game.")
            break
       else:
            game_should_quit = False

    # Add a small delay to make the movement visible, if desired
    # import time
    # time.sleep(0.1)

Generated first floor with 'E' at (1, 1).

--- Character Creation ---
Enter your hero's name: Matt
Choose your race: (H for Human, E for Elf, D for Dwarf)
> H
Choose your gender: (M for Male, F for Female, N for Non-binary)
> M

Welcome, Matt the male human Adventurer!
Health: 100, Attack: 15, Defense: 5, Strength: 10, Dexterity: 10, Intelligence: 10

--- Welcome to the Starting Shop! ---
Here you can purchase essential gear before venturing into the tower.
Today's merchant is Old Man Tiberius.
You are in a 'V' room. A friendly vendor greets you.

Your gold: 500
Vendor's wares:
  1. Iron Sword (Value: 79 gold) +0
  2. Healing Potion (Value: 23 gold)
  3. Greater Healing Potion (Value: 43 gold)
  4. Flare (Value: 15 gold)
  5. Lantern (Value: 27 gold)

Your inventory:
Inventory is empty.
Equipped Weapon: None
Equipped Armor: None
Enter 'b [number]' to buy, 's [number]' to sell, 'f' to finish shopping, or 'q' to quit: f
You finish shopping and prepare for your adventure!
    #           